In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

This notebook is a companion to the paper TBC, and reproduces the plots from the paper. We use the Ginibre point process as a running example. It is convenient, since its two-point correlation function and structure factor are analytically known.

In [ ]:
import numpy as np
from structure_factor.data import load_data
from structure_factor.structure_factor import StructureFactor
import structure_factor.utils as utils

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=14)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

# 1- Ginibre Ensemble 

In [ ]:
# load points
import pickle
path = "/Users/dhawat/github/structure_factor_paper/test_data/list_samples_ginibre_pp.pkl"
with open(path, "rb") as input_file:
    list_sample_ginibre_pp = pickle.load(input_file)

In [ ]:
from structure_factor.spatial_windows import BallWindow, BoxWindow
list_ginibre_pp_ball = list_sample_ginibre_pp["ginibre_pp_ball"]
ginibre_pp_ball = list_ginibre_pp_ball[2]
ginibre_pp_ball.plot(window_res=BallWindow(radius=50, center=(0,0)), file_name="ginibre_pp_ball.pdf" )

list_ginibre_pp_box = list_sample_ginibre_pp["ginibre_pp_box"]
ginibre_pp_box = list_ginibre_pp_box[2]
ginibre_pp_box.plot(file_name="ginibre_pp_box.pdf")

## 1.1- Direct spectral estimators (DSE)

In [ ]:
from structure_factor.spatial_windows import BoxWindow

sf_ginibre_box = StructureFactor(ginibre_pp_box) # initialize a new instance of StructureFactor
#ginibre_pp_box.plot(file_name="ginibre_sample.png")

### 1.1.1- Scattering intensity

In [ ]:

# on allowed values 
k_max=6
allowed_k, si = sf_ginibre_box.scattering_intensity(k_max=k_max)
print("Scattering intensity")


In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
sf_ginibre_box.plot_spectral_estimator(allowed_k, si, axes=axis, plot_type="radial",exact_sf=utils.structure_factor_ginibre,
                                            bins=60, error_bar=True,  label=r"$\widehat{S}$",
                                           file_name="si_allowed_k.pdf"
                                          )
plt.show()

Scattering intensity on non allowed values and debiased scattering intensity

In [ ]:
x = np.linspace(0, k_max, 150)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# on k 
k, si1 = sf_ginibre_box.scattering_intensity(k=k, debiased=False)
# on k debiased directly
k, si2 = sf_ginibre_box.scattering_intensity(k=k, debiased=True)
# on k debiased undirectly
k, si3 = sf_ginibre_box.scattering_intensity(k=k, debiased=True, direct=False)

In [ ]:
import matplotlib.pyplot as plt
fig, axis = plt.subplots(figsize=(7,6))
sf_ginibre_box.plot_spectral_estimator(k, si3, axes=axis, positive=False, plot_type="radial", exact_sf=utils.structure_factor_ginibre,
                                            bins=100, error_bar=True, label=r"$\widehat{S}$", rasterized=True,
                                           file_name="si_k_uddebiased_ginibre.pdf"
                                          )
plt.show()

 all the above estimators in one picture

In [ ]:
# save results 
import pickle
my_dict2 = {"ginibre_s_si_allowed_k": [allowed_k, si], "ginibre_s_si_k": [k, si1], "ginibre_s_si_directly_debiased": [k, si2],
            "ginibre_s_si_undirectly_debiased": [k, si3]}
with open('ginibre_s_si.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

In [ ]:
norm_k_allowed = utils.norm_k(allowed_k)
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(8, 6))
utils.plot_summary(norm_k_allowed, si, axis=axis, label=r"$\widehat{S}_{\mathrm{SI}}(2 \pi \mathbf{n}/ L)$", fmt='b', bins=60)
utils.plot_summary(norm_k, si1, axis=axis, label=r"$\widehat{S}_{\mathrm{SI}}(\mathbf{k})$", fmt='grey', bins=60 )
utils.plot_summary(norm_k, si2, axis=axis, label=r"$\widehat{S}_{\mathrm{DTP}}(\mathbf{k})$", fmt='k', bins=150 )
utils.plot_summary(norm_k, si3, axis=axis, label=r"$\widehat{S}_{\mathrm{UDTP}}(\mathbf{k})$",fmt='c', bins=180 )
axis.plot(norm_k, utils.structure_factor_ginibre(norm_k), 'g.' )

### 1.1.2- Tapered periodogram with sine taper

In [ ]:
from structure_factor.tapers import SineTaper
## tapered periodogram
x = np.linspace(0, k_max, 150)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
p = [1, 1]
taper = SineTaper(p)
k, s_tp = sf_ginibre_box.tapered_periodogram(k, taper, debiased=False)
k, s_dtp = sf_ginibre_box.tapered_periodogram(k, taper)
k, s_udtp = sf_ginibre_box.tapered_periodogram(k, taper, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_ginibre_box.plot_spectral_estimator(k, s_tp, axes=axis, positive=False, plot_type="radial",exact_sf=utils.structure_factor_ginibre,
                                            bins=100, error_bar=True, label=r"$\widehat{S}$",
                                           file_name="s_tp_sine_taper_ginibre.pdf"
                                          )

In [ ]:
# save results 
import pickle
my_dict2 = {"ginibre_s_tp_sine_taper": [k, s_tp], "ginibre_s_ddtp_sine_taper": [k, s_dtp], "ginibre_s_udtp_sine_taper": [k, s_udtp]}
with open('ginibre_s_tp_sine_taper.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

In [ ]:
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(8, 6))
utils.plot_summary(norm_k, s_tp, axis=axis, label=r"$\widehat{S}_{\mathrm{TP}}(\mathbf{k})$", fmt='b', bins=60 )
utils.plot_summary(norm_k, s_dtp, axis=axis, label=r"$\widehat{S}_{\mathrm{DTP}}(\mathbf{k})$", fmt='k', bins=150 )
utils.plot_summary(norm_k, s_udtp, axis=axis, label=r"$\widehat{S}_{\mathrm{UDTP}}(\mathbf{k})$",fmt='c', bins=180 )
axis.plot(norm_k, utils.structure_factor_ginibre(norm_k), 'g' )

 Result with scattering intensity

In [ ]:
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(8, 6))
utils.plot_summary(norm_k, s_dtp, axis=axis, label=r"$\widehat{S}_{\mathrm{DTP}}(\mathbf{k})$", fmt='c', bins=150 )
utils.plot_summary(norm_k_allowed, si, axis=axis, label=r"$\widehat{S}_{\mathrm{SI}}(2 \pi \mathbf{n}/ L)$", fmt='b', bins=60)
utils.plot_summary(norm_k, si2, axis=axis, label=r"$\widehat{S}_{\mathrm{SIDTP}}(\mathbf{k})$", fmt='k', bins=150 )
axis.plot(norm_k, utils.structure_factor_ginibre(norm_k), 'g' )

sine taper looks better than scattering inetsnity

### 1.1.3- Multitapering

In [ ]:
from structure_factor.tapers import SineTaper
## multitapered tapered periodogram
x = np.linspace(0, k_max, 150)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
k, s_mtp = sf_ginibre_box.multitapered_periodogram(k, debiased=False)
k, s_mdtp = sf_ginibre_box.multitapered_periodogram(k)
k, s_mudtp = sf_ginibre_box.multitapered_periodogram(k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_ginibre_box.plot_spectral_estimator(k, s_mtp, axes=axis, positive=False, plot_type="radial",exact_sf=utils.structure_factor_ginibre,
                                            bins=100, error_bar=True, label=r"$\widehat{S}$",
                                           file_name="s_mtp_sine_taper_ginibre.pdf"
                                          )

In [ ]:
# save results 
import pickle
my_dict2 = {"ginibre_s_mtp_sine_taper": [k, s_mtp], "ginibre_s_ddmtp_sine_taper": [k,s_mdtp], "ginibre_s_udmtp_sine_taper": [k, s_mudtp]}
with open('ginibre_s_mtp_sine_taper.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

In [ ]:
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(8, 6))
utils.plot_summary(norm_k, s_mtp, axis=axis, label=r"$\widehat{S}_{\mathrm{MTP}}(\mathbf{k})$", fmt='b', ecolor="b", bins=60 )
utils.plot_summary(norm_k, s_mdtp, axis=axis, label=r"$\widehat{S}_{\mathrm{MDTP}}(\mathbf{k})$", fmt='k', ecolor="k", bins=150 )
utils.plot_summary(norm_k, s_mudtp, axis=axis, label=r"$\widehat{S}_{\mathrm{MUDTP}}(\mathbf{k})$",fmt='c', ecolor="c", bins=180 )
axis.plot(norm_k, utils.structure_factor_ginibre(norm_k), 'g' )

compaising with the best of the above

In [ ]:
norm_k = utils.norm_k(k)
norm_k_allowed = utils.norm_k(allowed_k)

fig, axis = plt.subplots(figsize=(10, 8))
utils.plot_summary(norm_k_allowed, si, axis=axis, label=r"$\widehat{S}_{\mathrm{SI}}(2 \pi \mathbf{n}/ L)$", fmt='b', ecolor='b', bins=50)
#utils.plot_summary(norm_k, si2, axis=axis, label=r"$\widehat{S}_{\mathrm{SIDTP}}(\mathbf{k})$", fmt='k', ecolor='k', bins=150 )
utils.plot_summary(norm_k, s_dtp, axis=axis, label=r"$\widehat{S}_{\mathrm{DTP}}(\mathbf{k})$", fmt='k', ecolor='k', bins=50 )
utils.plot_summary(norm_k, s_mdtp, axis=axis, label=r"$\widehat{S}_{\mathrm{MDTP}}(\mathbf{k})$", fmt='m', ecolor="m", bins=50 )
axis.plot(norm_k, utils.structure_factor_ginibre(norm_k), 'g' )

## 1.2- Isotropic estimator (ISD)

In [ ]:
sf_ginibre_ball = StructureFactor(ginibre_pp_ball)

### 1.2.1- Bartlett isotropic estimator

In [ ]:
import time
start_time = time.time()
k_norm1, s_bi = sf_ginibre_ball.bartlett_isotropic_estimator(n_allowed_k_norm=100)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from structure_factor.statistics import get_k_norm
k_norm1= get_k_norm(ginibre_pp_ball, n_allowed_k_norm=100)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
sf_ginibre_ball.plot_isotropic_estimator(k_norm, s_bi, exact_sf=utils.structure_factor_ginibre, axis=axis,
                                    label=r"$\widehat{S}$", 
                                    file_name="s_bi_ginibre.pdf"
                                   )

In [ ]:
k_non_allowed = np.linspace(min(k_norm) + 0.002, max(k_norm), 100)
k_norm_non_allowed, s_bi_non_allowed = sf_ginibre_ball.bartlett_isotropic_estimator(k_non_allowed)


In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
sf_ginibre_ball.plot_isotropic_estimator(k_norm_non_allowed, s_bi_non_allowed, axis=axis, exact_sf=utils.structure_factor_ginibre,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_ginibre_non_allowed.pdf")

### 1.2.2- Hankel transform estimation

#### 1.2.2.1- Pair correlation function

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

pcf.ppp

In [ ]:
r= np.linspace(0, 20, 200)
ginibre_pcf_ppp = pcf.estimate(ginibre_pp_ball, method="ppp", r=r, correction="all")

In [ ]:
pcf.plot(ginibre_pcf_ppp, exact_pcf=utils.pair_correlation_function_ginibre, figsize=(7,6),
         color=['grey', 'b', 'darkcyan'], style=[".", "*", "^"],
         file_name="pcf_ppp_ginibre.pdf")

pcf.fv

In [ ]:
ginibre_pcf_fv = pcf.estimate(ginibre_pp_ball, method="fv", Kest=dict(rmax=20), fv=dict(method="b", spar=0.2))
ginibre_pcf_fv

In [ ]:
pcf.plot(ginibre_pcf_fv, exact_pcf=utils.pair_correlation_function_ginibre, figsize=(7,6), color=['grey'], style=["."],
         file_name="pcf_fv_ginibre.pdf"
        )

Interpolation

In [ ]:
r = ginibre_pcf_fv["r"]
r_max = np.max(r)
pcf_r = ginibre_pcf_fv["pcf"]
# interpolation
ginibre_pcf_fct = pcf.interpolate(r=r, pcf_r=pcf_r, drop=True)


x = np.linspace(0, 100, 200)
plt.plot(x, ginibre_pcf_fct(x), 'b.')

plt.plot(x, utils.pair_correlation_function_ginibre(x), 'g')

Ogata

In [ ]:
k_norm2 = np.linspace(0.3, 30, 1000) # vector of wave length
k_norm2, s_ho = sf_ginibre_ball.hankel_quadrature(ginibre_pcf_fct, method="Ogata", k_norm=k_norm2, step_size=0.01,
                                                    nb_points=1000)
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_ginibre_ball.plot_isotropic_estimator(k_norm2, s_ho, axis=axis,error_bar=True, bins=40, exact_sf=utils.structure_factor_ginibre, 
                                               label=r"$\widehat{S}$",
                                               #file_name="ginibre_s_ho.pdf"
                                              )

In [ ]:
# from exact pcf
k_norm = np.linspace(0.001, 30, 1000) # vector of wave length
k_norm, s_ho = sf_ginibre.hankel_quadrature(utils.pair_correlation_function_ginibre, method="Ogata", k_norm=k_norm, step_size=0.00001,
                                                    nb_points=500)
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_ginibre.plot_isotropic_estimator(k_norm, s_ho, axis=axis,error_bar=False, bins=30, exact_sf=utils.structure_factor_ginibre, label=r"$\widehat{S}_{\mathrm{HO}}(k)$",
                                          file_name="ginibre_s_ho_exact.png"
                                           )


Baddour-Chouinard

In [ ]:
r_max

In [ ]:
k = np.linspace(0.3, 30, 1000)
k_norm3, s_hbc = sf_ginibre_ball.hankel_quadrature(ginibre_pcf_fct,k_norm=k,  r_max=r_max,  nb_points=3000 )

In [ ]:
fig, axis= plt.subplots(figsize=(7,6))
sf_ginibre_ball.plot_isotropic_estimator(k_norm3, s_hbc, axis=axis, exact_sf=utils.structure_factor_ginibre, 
                                    error_bar=True, bins=100,
                                    label=r"$\widehat{S}$",
                                    file_name="ginibre_s_hbc.pdf"
                                   )

In [ ]:
# save results 
import pickle
my_dict2 = {"ginibre_s_bi": [k_norm1, s_bi], "ginibre_s_ho": [k_norm2, s_ho], "ginibre_s_hbc":[k_norm3, s_hbc] }
with open('ginibre_s_ise.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

In [ ]:
k = np.linspace(0, 10, 400)
k_norm, s_hbc = sf_ginibre.hankel_quadrature(utils.pair_correlation_function_ginibre,k_norm=k,  r_max=r_max,  nb_points=1000 )

In [ ]:
fig, axis= plt.subplots(figsize=(7,6))
sf_ginibre.plot_isotropic_estimator(k_norm, s_hbc, axis=axis, exact_sf=utils.structure_factor_ginibre, error_bar=False, bins=40,
                                    label=r"$\widehat{S}_{\mathrm{HBC}}(k)$",file_name="ginibre_s_hbc_exact.png")

## 1.3- Hyperuniformity

In [ ]:
from structure_factor.hyperuniformity import Hyperuniformity
hyperuniformity_test = Hyperuniformity(k_norm, s_bi)
#bin_center, bin_mean, bin_std = hyperuniformity_test.bin_data(bins=80)

### 1.3.1- Effective hyperuniformity

In [ ]:
H_ginibre, std = hyperuniformity_test.effective_hyperuniformity(k_norm_stop=1)
print("H_ginibre=", H_ginibre)

In [ ]:
import matplotlib.pyplot as plt
#k_norm = hyperuniformity_test.k_norm
#sf = hyperuniformity_test.sf
sf = s_bi
fitted_sf_line = hyperuniformity_test.fitted_line # fitted line to sf
index_peak = hyperuniformity_test.i_first_peak # index of first peak of sf
x = np.linspace(0, 5, 300)
y = np.linspace(0,np.max(k_norm), 500)
fig=plt.figure(figsize=(10,6))
plt.plot(k_norm, sf, 'b.', label="approx_sf")
plt.plot(k_norm, sf, 'b', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.plot(y, utils.structure_factor_ginibre(y), 'g', label="exact sf")
#plt.plot(mean_k_norm[index_peak], mean_sf[index_peak], 'k*', label="first peak")
plt.legend()
plt.xlabel('wavelength ($||\mathbf{k}||$)')
plt.ylabel('Structure factor ($\mathsf{S}(\mathbf{k})$)')
plt.show()
#fig.savefig("fitted_si_ginibre.pdf", bbox_inches="tight")

### 1.3.2- Hyperuniformity class

In [ ]:
sf_power_decay, c = hyperuniformity_test.hyperuniformity_class(k_norm_stop=0.4)
print(sf_power_decay)

In [ ]:
import matplotlib.pyplot as plt
#k_norm = hyperuniformity_test.k_norm
#sf = hyperuniformity_test.sf
sf = s_bi
fitted_poly = hyperuniformity_test.fitted_poly
x = np.linspace(0, 2, 300)
y = np.linspace(0,np.max(k_norm), 500)
fig=plt.figure(figsize=(10,6))
plt.plot(k_norm, sf, 'b.', label="approx_sf")
plt.plot(k_norm, sf, 'b.', label="approx_sf")

plt.plot(k_norm, sf, 'b', label="approx_sf")
plt.plot(y, utils.structure_factor_ginibre(y), 'g', label="exact sf")
plt.plot(x, fitted_poly(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength ($||\mathbf{k}||$)')
plt.ylabel('Structure factor ($\mathsf{S}(\mathbf{k})$)')
plt.show()
#fig.savefig("fitted_si_ginibre.pdf", bbox_inches="tight")

## 1.4- Statistics

Load samples of points

In [ ]:
# load points
import pickle
path = "/Users/dhawat/github/structure_factor_paper/test_data/list_samples_ginibre_pp.pkl"
with open(path, "rb") as input_file:
    list_sample_ginibre_pp = pickle.load(input_file)
    
from structure_factor.spatial_windows import BallWindow, BoxWindow
ginibre_ball_pp_list = list_sample_ginibre_pp["ginibre_pp_ball"]
ginibre_ball_pp_list[0].plot(window_res=BallWindow(radius=50, center=(0,0)) )

ginibre_box_pp_list = list_sample_ginibre_pp["ginibre_pp_box"]
ginibre_box_pp_list[0].plot()

Initialize statistics

In [ ]:
# statistics
from structure_factor.statistics import SummaryStatistics, get_k, get_k_norm

ginibre_ball_statistics = SummaryStatistics(ginibre_ball_pp_list)
ginibre_box_statistics = SummaryStatistics(ginibre_box_pp_list)

### 1.4.1- For Direct spectral estimators 

In [ ]:
from structure_factor.statistics import apply_estimator
from multiprocessing import Pool
from functools import partial
from structure_factor.tapers import SineTaper
import time

start_time = time.time()
core_num=8

k_allowed = get_k(ginibre_box_pp_list[0], k_max=2)
#x_neg = np.linspace(-np.max(np.abs(k_allowed)), -np.min(np.abs(k_allowed)), int(46/2))
x = np.linspace(np.min(np.abs(k_allowed)), np.max(np.abs(k_allowed)), 30)
#x = np.row_stack((x_neg, x_pos)).ravel()
#x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
p = [1, 1]
taper = SineTaper(p)

norm_k_allowed1 = utils.norm_k(k_allowed)
norm_k = utils.norm_k(k)

In [ ]:
print(np.max(norm_k_allowed1), np.max(norm_k), np.min(norm_k_allowed1), np.min(norm_k))

In [ ]:
if __name__ == "__main__":
    # scattering intensity
    s_si_allowed_k = ginibre_box_statistics.sample_approximation(estimator="scattering_intensity", k=k_allowed)
    # scattering intensity debiased
    s_dtp_t0 = ginibre_box_statistics.sample_approximation(estimator="scattering_intensity", k=k, debiased=True, direct=True)
    # debiased tapered periodogram (sinusoidal taper)
    s_dtp_t1 = ginibre_box_statistics.sample_approximation(estimator="tapered_periodogram", k=k,taper=taper, debiased=True, direct=True)
    # multitapered periodogram (sinusoidal taper)
    s_mdtp = ginibre_box_statistics.sample_approximation(estimator="multitapered_periodogram", k=k, debiased=True, direct=True)
    
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
from structure_factor.statistics import list_vertical_sample_mean
norm_k_unique, s_si_allowed_k_mean = list_vertical_sample_mean(norm_k_allowed1,s_si_allowed_k)

In [ ]:
plt.loglog(norm_k_allowed1,s_si_allowed_k[1], 'b*' )
plt.loglog(norm_k_unique, s_si_allowed_k_mean[1], 'r.')
plt.loglog(norm_k_unique, utils.structure_factor_ginibre(norm_k_unique), 'g')

In [ ]:
norm_k_allowed1 = utils.norm_k(k_allowed)
norm_k = utils.norm_k(k)
#print("first 20 allowed k:", utils.norm_k(k_allowed[:20]))
#print("last 20 allowed k:", utils.norm_k(k_allowed[-20:]))
sorted_norm_k_allowed, sorted_s_si, _= utils._sort_vectors(norm_k_allowed1, s_si_allowed_k[0])
print("first 30 norm allowed k sorted:", sorted_norm_k_allowed[:30] )
print("first 30 sorted s_si:", sorted_s_si[:30])

Dont touch above

In [ ]:
from structure_factor.statistics import sample_statistics
k_norm_si, m_si, ivar_si, ibias_si, imse_si, imse_var  = sample_statistics(k=k_allowed,
                                                                                approximation=s_si_allowed_k,
                                                                                exact=utils.structure_factor_ginibre )

In [ ]:
k[:, 1:].shape

In [ ]:
from structure_factor.statistics import sample_statistics

k_norm_si, m_si, ivar_si, ibias_si, imse_si, imse_var_si = sample_statistics(k=k_allowed,
                                                                        approximation=s_si_allowed_k,
                                                                                exact=utils.structure_factor_ginibre )
k_norm_dtp_t0, m_dtp_t0, ivar_dtp_t0, ibias_dtp_t0, imse_dtp_t0, imse_var_dtp_t0 = sample_statistics(
                                                                                k=k,
                                                                                approximation=s_dtp_t0,
                                                                                exact=utils.structure_factor_ginibre )
k_norm_dtp_t1, m_dtp_t1, ivar_dtp_t1, ibias_dtp_t1, imse_dtp_t1, imse_var_dtp_t1 = sample_statistics(k=k,
                                                                                approximation=s_dtp_t1,
                                                                                exact=utils.structure_factor_ginibre )
k_norm_mdtp, m_mdtp, ivar_mdtp, ibias_mdtp, imse_mdtp, imse_var_mdtp = sample_statistics(k=k,
                                                                                approximation=s_mdtp,
                                                                                exact=utils.structure_factor_ginibre )
print("For s_si: ivar", ivar_si, "ibias", ibias_si, "imse", imse_si, "3std(imse)",  np.sqrt(imse_var_si)*3)
print("For s_dtp_t0: ivar", ivar_dtp_t0, "ibias", ibias_dtp_t0, "imse", imse_dtp_t0, "3std(imse)",  np.sqrt(imse_var_dtp_t0)*3)
print("For s_dtp_t1: ivar", ivar_dtp_t1, "ibias", ibias_dtp_t1, "imse", imse_dtp_t1, "3std(imse)",  np.sqrt(imse_var_dtp_t1)*3)
print("For s_mdtp: ivar", ivar_mdtp, "ibias", ibias_mdtp, "imse", imse_mdtp, "3std(imse)",  np.sqrt(imse_var_mdtp)*3)

In [ ]:
norm_k_allowed1 = utils.norm_k(k_allowed)
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(7,6))

axis.loglog(k_norm_si, m_si, 'b^', label=r"$\overline{\widehat{S}_{\mathrm{SI}}(2\pi \mathbf{n}/L)}$")
#axis.errorbar(k_norm_si, m_si, yerr=var_si, lw=1, ecolor='b', capsize=3, capthick=1)
axis.loglog(k_norm_dtp_t0, m_dtp_t0, 'k*', label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_0, \mathbf{k})}$")
#axis.errorbar(k_norm_dtp_t0, m_dtp_t0, yerr=var_dtp_t0, lw=1, ecolor='k', capsize=3, capthick=1)
axis.loglog(k_norm_dtp_t1, m_dtp_t1, 'm+',label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_1, \mathbf{k})}$")
#axis.errorbar(k_norm_dtp_t1, m_dtp_t1, yerr=var_dtp_t1, lw=1, ecolor='m', capsize=3, capthick=1)
axis.loglog(k_norm_mdtp, m_mdtp, 'c.',  label=r"$\overline{\widehat{S}_{\mathrm{DDMTP}}((t_j)_{j=1}^4, \mathbf{k})}$")
utils.plot_exact(norm_k_allowed1, utils.structure_factor_ginibre(norm_k_allowed1), axis=axis, label=r"$S(\mathbf{k})$")
axis.legend()
axis.set_xlabel(r"Wavenumber ($k$)")
axis.set_ylabel(r"Structure factor ($S(k)$)")
fig.savefig("comparaison_DSE_mean_ginibre.pdf", bbox_inches="tight")

In [ ]:
norm_k_allowed1 = utils.norm_k(k_allowed)
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(7,6))

axis.loglog(norm_k_allowed1,m_si, 'b^', label=r"$\overline{\widehat{S}_{\mathrm{SI}}(2\pi \mathbf{n}/L)}$")
axis.loglog(norm_k,m_dtp_t0, 'k*', label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_0, \mathbf{k})}$")
axis.loglog(norm_k,m_dtp_t1, 'm+',label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_1, \mathbf{k})}$")
axis.loglog(norm_k,m_mdtp, 'c.',  label=r"$\overline{\widehat{S}_{\mathrm{MDDTP}}((t_j)_{j=1}^4, \mathbf{k})}$")
utils.plot_exact(norm_k_allowed1, utils.structure_factor_ginibre(norm_k_allowed1), axis=axis, label=r"$S(\mathbf{k})$")
axis.legend()
axis.set_xlabel(r"Wavenumber ($||\mathbf{k}||$)")
axis.set_ylabel(r"Structure factor ($S(\mathbf{k})$)")
fig.savefig("comparaison_DSE_mean_ginibre.png", bbox_inches="tight")

In [ ]:
s_mdtp==None

In [ ]:
norm_k_allowed1 = utils.norm_k(k_allowed)
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(7,6))

utils.plot_summary_2(norm_k_allowed1,m_si, c='b',marker='*', bins=30, axis=axis, label=r"$\overline{\widehat{S}_{\mathrm{SI}}(2\pi \mathbf{n}/L)}$")
utils.plot_summary_2(norm_k,m_dtp_t0, c='k',marker='.', bins=30, axis=axis, label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_0, \mathbf{k})}$")
utils.plot_summary_2(norm_k,m_dtp_t1, c='m',marker='+', bins=30, axis=axis, label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_1, \mathbf{k})}$")
utils.plot_summary_2(norm_k,m_mdtp, c='grey',marker='o', bins=30, axis=axis, label=r"$\overline{\widehat{S}_{\mathrm{MDDTP}}((t_j)_{j=1}^4, \mathbf{k})}$")
axis.loglog(norm_k_allowed1, utils.structure_factor_ginibre(norm_k_allowed1), 'g', label=r"$S(\mathbf{k})$")
axis.legend()
axis.set_xlabel(r"Wavenumber ($||\mathbf{k}||$)")
axis.set_ylabel(r"Structure factor ($S(\mathbf{k})$)")
fig.savefig("comparaison_DSE_mean_ginibre.png", bbox_inches="tight")

### 1.4.2- For isotropic estimators

In [ ]:
from structure_factor.statistics import apply_estimator
from multiprocessing import Pool
from functools import partial
from structure_factor.statistics import pcf_interpolate_list
import time

start_time = time.time()
core_num=8

r_max=30
n_allowed_k_norm=64
k_norm_allowed = get_k_norm(ginibre_ball_pp_list[0], n_allowed_k_norm=n_allowed_k_norm)

if __name__ == "__main__":
    # Bartlett's isotropic estimator
    s_bi = ginibre_ball_statistics.sample_approximation(estimator="bartlett_isotropic_estimator", n_allowed_k_norm=n_allowed_k_norm)
    # list pcf
    #pcf_list = ginibre_ball_statistics.sample_pcf_approximation(method="fv", Kest=dict(rmax=60, correction="Ripley"), fv=dict(method="b", spar=0.1))
    pcf_list = ginibre_ball_statistics.sample_pcf_approximation(method="fv", Kest=dict(rmax=r_max), fv=dict(method="b", spar=0.2))
    
print("--- %s seconds ---" % (time.time() - start_time))



In [ ]:
s = len(pcf_list)
# interpolate pcf
r_max=30
r_list = [pcf_list[i]["r"] for i in range(s)]
pcf_fv_list = [pcf_list[i]["pcf"] for i in range(s)]
pcf_fct_list = pcf_interpolate_list(r_list, pcf_fv_list, drop=True)
# BaddourChouinard
s_hbc_list = ginibre_ball_statistics.sample_integral_approximation(pcf_fct_list,method="BaddourChouinard", 
                                                                   k_norm=k_norm_allowed, r_max=r_max,  nb_points=1000) 
# Ogata
s_ho_list = ginibre_ball_statistics.sample_integral_approximation(pcf_fct_list,method="Ogata", 
                                                                   k_norm=k_norm_allowed, step_size=0.01,  nb_points=1000) 

In [ ]:
# save results 
import pickle
my_dict2 = {"ginibre_s_bi_list": [k_norm_allowed, s_bi], "ginibre_s_ho_list": [k_norm_allowed, s_ho_list],
            "ginibre_s_hbc_list": [k_norm_allowed, s_hbc_list]}
with open('ginibre_ise_list.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

In [ ]:
k_bi, m_bi, ivar_bi, ibias_bi, imse_bi, imse_var_bi = sample_statistics(k_norm=k_norm_allowed,
                                                                               approximation=s_bi,
                                                                               exact=utils.structure_factor_ginibre )
k_bi, m_hbc, ivar_hbc, ibias_hbc, imse_hbc, imse_var_hbc =sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_hbc_list,
                                                                                exact=utils.structure_factor_ginibre )
k_bi, m_ho, ivar_ho, ibias_ho, imse_ho, imse_var_ho = sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_ho_list,
                                                                                exact=utils.structure_factor_ginibre )
print("For s_bi: ivar=", ivar_bi, "ibias=", ibias_bi, "imse=", imse_bi, "3std(imse)",  np.sqrt(imse_var_bi)*3)

print("For s_hbc: ivar=", ivar_hbc, "ibias=", ibias_hbc, "imse=", imse_hbc, "3std(imse)",  np.sqrt(imse_var_hbc)*3)

print("For s_ho: ivar=", ivar_ho, "ibias=", ibias_ho, "imse=", imse_ho, "3std(imse)",  np.sqrt(imse_var_ho)*3)

In [ ]:
k_norm_allowed

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
axis.loglog(k_norm_allowed, m_bi, 'm*', label=r"$\overline{\widehat{S}_{\mathrm{BI}}(k)}$")
#axis.loglog(k_norm_allowed, m_bi, 'm')
axis.loglog(k_ho, m_ho, 'b.', label=r"$\overline{\widehat{S}_{\mathrm{HO}}(k)}$")
#axis.loglog(k_norm_allowed, m_ho, 'b')
axis.loglog(k_hbc, m_hbc, 'k+', label=r"$\overline{\widehat{S}_{\mathrm{HBC}}(k)}$")
#axis.loglog(k_norm_allowed, m_hbc, 'k')
axis.loglog(k_norm_allowed, utils.structure_factor_ginibre(k_norm_allowed), 'g', label= r"$S(k)$")
axis.legend()
axis.set_xlabel(r"Wavenumber ($k$)")
axis.set_ylabel(r"Structure factor ($S(k)$)")
plt.show()
fig.savefig("comparaison_ISE_mean_ginibre.pdf", bbox_inches="tight")

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf
pcf.plot(pcf_list[0], exact_pcf=utils.pair_correlation_function_ginibre, figsize=(7,6),
         color=['grey', 'b', 'darkcyan'], style=[".", "*", "^"],
         file_name="pcf_ppp_ginibre.png")

# trying to see the effect of fixing interpolation limite on variance badddour ogata

In [ ]:
pcf

In [ ]:
s = len(pcf_list)
r_list_ = [pcf_list[i]["r"] for i in range(s)] 
r_max = np.max(r_list_[0])
r_list2 = [np.append(r_list_[i], r_max + 800) for i in range(s)]
pcf_r_list_ = [pcf_list[i]["pcf"] for i in range(s)]
pcf_list2 = [np.append(pcf_r_list_[i], 1) for i in range(s)]

In [ ]:
# interpolate pcf
pcf_fct_list = pcf_interpolate_list(r_list2, pcf_list2, drop=True)
plt.plot(r_list2[0], pcf_fct_list[0](r_list2[0]) , 'b')

In [ ]:
# BaddourChouinard
s_hbc_list = ginibre_ball_statistics.sample_integral_approximation(pcf_fct_list,method="BaddourChouinard", 
                                                                   k_norm=k_norm_allowed, r_max=0.5,  nb_points=1000) 
# Ogata
s_ho_list = ginibre_ball_statistics.sample_integral_approximation(pcf_fct_list,method="Ogata", 
                                                                   k_norm=k_norm_allowed, step_size=0.02,  nb_points=1000) 

In [ ]:
#m_bi, var_bi, ivar_bi, bias_bi, ibias_bi, mse_bi, imse_bi = ginibre_ball_statistics.sample_statistics(k_norm=k_norm_allowed,
#                                                                                approximation=s_bi,
#                                                                                exact=utils.structure_factor_ginibre )
m_hbc, var_hbc, ivar_hbc, bias_hbc, ibias_hbc, mse_hbc, imse_hbc = ginibre_ball_statistics.sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_hbc_list,
                                                                                exact=utils.structure_factor_ginibre )
m_ho, var_ho, ivar_ho, bias_ho, ibias_ho, mse_ho, imse_ho = ginibre_ball_statistics.sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_ho_list,
                                                                                exact=utils.structure_factor_ginibre )
#print("For s_bi: ivar=", ivar_bi, "ibias=", ibias_bi, "imse=", imse_bi)

print("For s_hbc: ivar=", ivar_hbc, "ibias=", ibias_hbc, "imse=", imse_hbc)

print("For s_ho: ivar=", ivar_ho, "ibias=", ibias_ho, "imse=", imse_ho)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
#axis.loglog(k_norm_allowed, m_bi, 'm*', label=r"$\overline{\widehat{S}_{\mathrm{BI}}(k)}$")
#axis.loglog(k_norm_allowed, m_bi, 'm')
axis.plot(k_norm_allowed, m_ho, 'b.', label=r"$\overline{\widehat{S}_{\mathrm{HO}}(k)}$")
#axis.loglog(k_norm_allowed, m_ho, 'b')
axis.plot(k_norm_allowed, m_hbc, 'k+', label=r"$\overline{\widehat{S}_{\mathrm{HBC}}(k)}$")
#axis.loglog(k_norm_allowed, m_hbc, 'k')
axis.loglog(k_norm_allowed, utils.structure_factor_ginibre(k_norm_allowed), 'g', label= r"$S(k)$")
axis.legend()
axis.set_xlabel(r"Wavenumber ($k$)")
axis.set_ylabel(r"Structure factor ($S(k)$)")
plt.show()
fig.savefig("comparaison_ISE_mean_ginibre.png", bbox_inches="tight")

In [ ]:
xx = np.linspace(0,2, 400)
plt.loglog(xx, utils.structure_factor_ginibre(xx))

# untill here

In [ ]:
s = len(pcf_list)
# interpolate pcf
r_list = [pcf_list[i]["r"] for i in range(s)]
pcf_fv_list = [pcf_list[i]["pcf"] for i in range(s)]
pcf_fct_list = pcf_interpolate_list(r_list, pcf_fv_list, drop=True)
# BaddourChouinard
s_hbc_list = ginibre_ball_statistics.sample_integral_approximation(pcf_fct_list,method="BaddourChouinard", 
                                                                   k_norm=k_norm_allowed, r_max=r_max,  nb_points=1000) 
# Ogata
s_ho_list = ginibre_ball_statistics.sample_integral_approximation(pcf_fct_list,method="Ogata", 
                                                                   k_norm=k_norm_allowed, step_size=0.01,  nb_points=500) 

In [ ]:
thomas_box_pp_list[2].window.bounds

In [ ]:
#m_bi, var_bi, ivar_bi, bias_bi, ibias_bi, mse_bi, imse_bi = ginibre_ball_statistics.sample_statistics(k_norm=k_norm_allowed,
#                                                                                approximation=s_bi,
#                                                                                exact=utils.structure_factor_ginibre )
m_hbc, var_hbc, ivar_hbc, bias_hbc, ibias_hbc, mse_hbc, imse_hbc = ginibre_ball_statistics.sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_hbc_list,
                                                                                exact=utils.structure_factor_ginibre )
m_ho, var_ho, ivar_ho, bias_ho, ibias_ho, mse_ho, imse_ho = ginibre_ball_statistics.sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_ho_list,
                                                                                exact=utils.structure_factor_ginibre )
#print("For s_bi: ivar=", ivar_bi, "ibias=", ibias_bi, "imse=", imse_bi)

print("For s_hbc: ivar=", ivar_hbc, "ibias=", ibias_hbc, "imse=", imse_hbc)

print("For s_ho: ivar=", ivar_ho, "ibias=", ibias_ho, "imse=", imse_ho)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
axis.loglog(k_norm_allowed, m_bi, 'm*', label=r"$\overline{\widehat{S}_{\mathrm{BI}}(k)}$")
axis.loglog(k_norm_allowed, m_bi, 'm')
axis.loglog(k_norm_allowed, m_ho, 'b.', label=r"$\overline{\widehat{S}_{\mathrm{HO}}(k)}$")
#axis.loglog(k_norm_allowed, m_ho, 'b')
axis.loglog(k_norm_allowed, m_hbc, 'k+', label=r"$\overline{\widehat{S}_{\mathrm{HBC}}(k)}$")
#axis.loglog(k_norm_allowed, m_hbc, 'k')
axis.loglog(k_norm_allowed, utils.structure_factor_ginibre(k_norm_allowed), 'g', label= r"$S(k)$")
axis.legend()
axis.set_xlabel(r"Wavenumber ($k$)")
axis.set_ylabel(r"Structure factor ($S(k)$)")
plt.show()
fig.savefig("comparaison_ISE_mean_ginibre.png", bbox_inches="tight")

## 1.4.3- Hyperuniformity study

The study will be build on the result of Bartlett's isotropic estimator

In [ ]:
from structure_factor.statistics import apply_estimator
from multiprocessing import Pool
from functools import partial
from structure_factor.statistics import pcf_interpolate_list
import time

start_time = time.time()
core_num=8

n_allowed_k_norm=64
k_norm_allowed = get_k_norm(ginibre_ball_pp_list[0], n_allowed_k_norm=n_allowed_k_norm)

if __name__ == "__main__":
    # Bartlett's isotropic estimator
    s_bi = ginibre_ball_statistics.sample_approximation(estimator="bartlett_isotropic_estimator", n_allowed_k_norm=n_allowed_k_norm)
    
print("--- %s seconds ---" % (time.time() - start_time))



In [ ]:
# save sample s_bi:
import pickle
my_dict = {"k_norm": k_norm_allowed, "list_s_bi_ginibre": s_bi}
with open('sample_s_bi_ginibre.pkl', 'wb') as f:
       pickle.dump(my_dict, f)

### 1.4.3.1- Effective hyperuniformity

In [ ]:
from structure_factor.statistics import sample_hyperuniformity, sample_variance

H_index_list = sample_hyperuniformity(k_norm_allowed, s_bi, estimator="effective", k_norm_stop=1)

In [ ]:
H_var, H_m = sample_variance(H_index_list)
print("mean H index", H_m, "3std H index", 3*np.sqrt(H_var))

### 1.4.3.2- Hyperuniformity class

In [ ]:
alpha_list = sample_hyperuniformity(k_norm_allowed, s_bi, estimator="class", k_norm_stop=0.45)
alpha_var, alpha_m = sample_variance(alpha_list)
print("mean power decay", alpha_m, "3std power decay", 3*np.sqrt(alpha_var))


# 2- Poisson point process

In this section we repeat the previous experience on a point configuration from a standard homogeneous Poisson Point Process in a cubic window

In [ ]:
# load points
import pickle
path = "/Users/dhawat/github/structure_factor_paper/test_data/list_samples_poisson_pp.pkl"
with open(path, "rb") as input_file:
    list_sample_poisson_pp = pickle.load(input_file)

In [ ]:
from structure_factor.spatial_windows import BallWindow, BoxWindow
list_poisson_pp_ball = list_sample_poisson_pp["poisson_pp_ball"]
poisson_pp_ball = list_poisson_pp_ball[2]
poisson_pp_ball.plot(window_res=BallWindow(radius=50, center=(0,0)), file_name="poisson_pp_ball.pdf" )

list_poisson_pp_box = list_sample_poisson_pp["poisson_pp_box"]
poisson_pp_box = list_poisson_pp_box[2]
poisson_pp_box.plot(file_name="poisson_pp_box.pdf")

## 2.1- Direct sprectral estimator

In [ ]:
sf_poisson_box = StructureFactor(poisson_pp_box) # initializing the StructureFactor class


### 2.1.2- Scattering intensity

In [ ]:
# on allowed values 
k_max = 6
allowed_k, si = sf_poisson_box.scattering_intensity(k_max=k_max)
print("Scattering intensity")
k_norm = np.linalg.norm(allowed_k, axis=1)
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_poisson_box.plot_spectral_estimator(allowed_k, si, axes=axis, plot_type="radial",
                                             #exact_sf=utils.structure_factor_poisson,
                                               label=r"$\widehat{S}$",
                                            bins=60, error_bar=True, file_name="si_allowed_k_pois.pdf")

In [ ]:
x = np.linspace(0, k_max, 150)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# on k 
k, si1 = sf_poisson_box.scattering_intensity(k=k, debiased=False)
# on k debiased directly
k, si2 = sf_poisson_box.scattering_intensity(k=k, debiased=True)
# on k debiased undirectly
k, si3 = sf_poisson_box.scattering_intensity(k=k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_poisson_box.plot_spectral_estimator(k, si3, axes=axis, positive=False, plot_type="radial",
                                             #exact_sf=utils.structure_factor_poisson,
                                               label=r"$\widehat{S}$",
                                            bins=65, error_bar=True, file_name="si_uddirect_pois.pdf")

In [ ]:
# save results 
import pickle
my_dict2 = {"poisson_s_si_allowed_k": [allowed_k, si], "poisson_s_si_k": [k, si1], "poisson_s_si_directly_debiased": [k, si2],
            "poisson_s_si_undirectly_debiased": [k, si3]}
with open('poisson_s_si.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

### 2.1.2- Tapered periodogram with SineTaper

In [ ]:
from structure_factor.tapers import SineTaper
## tapered periodogram
x = np.linspace(0, 6, 150)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
p = [1, 1]
taper = SineTaper(p)
k, s_tp = sf_poisson_box.tapered_periodogram(k, taper, debiased=False)
k, s_dtp = sf_poisson_box.tapered_periodogram(k, taper)
k, s_udtp = sf_poisson_box.tapered_periodogram(k, taper, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_poisson_box.plot_spectral_estimator(k, s_tp, axes=axis, positive=False, plot_type="radial",
                                            bins=60, error_bar=True, label=r"$\widehat{S}$",
                                           file_name="s_tp_sine_taper_poisson.pdf"
                                          )

In [ ]:
# save results 
import pickle
my_dict2 = {"poisson_s_tp_sine_taper": [k, s_tp], "poisson_s_ddtp_sine_taper": [k, s_dtp], "poisson_s_udtp_sine_taper": [k, s_udtp]}
with open('poisson_s_tp_sine_taper.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

### 2.1.3- Multitapering

In [ ]:
from structure_factor.tapers import SineTaper
## multitapered tapered periodogram
x = np.linspace(0, k_max, 150)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
k, s_mtp = sf_poisson_box.multitapered_periodogram(k, debiased=False)
k, s_ddmtp = sf_poisson_box.multitapered_periodogram(k, P=2)
k, s_udmtp = sf_poisson_box.multitapered_periodogram(k, debiased=True, direct=False, P=2)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_poisson_box.plot_spectral_estimator(k, s_udmtp, axes=axis, positive=False, plot_type="radial",
                                            bins=60, error_bar=True, label=r"$\widehat{S}$",
                                           file_name="s_udmtp_sine_taper_poisson.pdf"
                                          )

In [ ]:
# save results 
import pickle
my_dict2 = {"poisson_s_mtp_sine_taper": [k, s_mtp], "poisson_s_ddmtp_sine_taper": [k,s_mdtp], "poisson_s_udmtp_sine_taper": [k, s_mudtp]}
with open('poisson_s_mtp_sine_taper.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

## 2.2- Isotropic estimators

In [ ]:
sf_poisson_ball = StructureFactor(poisson_pp_ball)

### 2.2.1- Bartlett isotropic estimators

In [ ]:
n_allowed_k_norm=100
k_norm_allowed, s_bi = sf_poisson_ball.bartlett_isotropic_estimator(n_allowed_k_norm=n_allowed_k_norm)
fig, axis = plt.subplots(figsize=(7,6))
sf_poisson_ball.plot_isotropic_estimator(k_norm_allowed, s_bi, axis=axis,
                                         label=r"$\widehat{S}$", file_name="s_bi_poisson.pdf")

In [ ]:
sf_poisson_ball.plot_isotropic_estimator(k_norm, s_bi, axis=axis,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_poisson.png")

In [ ]:
k_non_allowed = np.linspace(min(k_norm)+0.002, max(k_norm) + 0.002, 100)
k_norm_non_allowed, s_bi_non_allowed = sf_poisson_ball.bartlett_isotropic_estimator(k_non_allowed)
fig, axis = plt.subplots(figsize=(7,6))
sf_poisson_ball.plot_isotropic_estimator(k_norm_non_allowed, s_bi_non_allowed, axis=axis,
                                         label=r"$\widehat{S}$", file_name="s_bi_poisson_non_allowed.pdf")

In [ ]:
sf_poisson_ball.plot_isotropic_estimator(k_norm_non_allowed, s_bi_non_allowed, axis=axis,
                                         label=r"$\widehat{S}_{\mathrm{BI}}(k)$", file_name="s_bi_poisson_non_allowed.png")
plt.show()

### 2.2.2- Pair correlation function

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

pcf.ppp

In [ ]:
r= np.linspace(0, 20, 200)
poisson_pcf_ppp = pcf.estimate(poisson_pp_ball, method="ppp", r=r, correction="all")
pcf.plot(poisson_pcf_ppp, figsize=(7,6),
         color=['grey', 'b', 'darkcyan'], style=[".", "*", "^"],
         file_name="pcf_ppp_poisson.pdf")

pcf.fv

In [ ]:
poisson_pcf_fv = pcf.estimate(poisson_pp_ball, method="fv", Kest=dict(rmax=20), fv=dict(method="c", spar=0.2))
pcf.plot(poisson_pcf_fv, figsize=(7,6),
         color=['grey'], style=["."],
         file_name="pcf_fv_poisson.pdf")

Interpolation

In [ ]:
r = poisson_pcf_fv["r"]
r_max = np.max(r)
pcf_r = poisson_pcf_fv["pcf"]
poisson_pcf_fct = pcf.interpolate(r=r, pcf_r=pcf_r, drop=True)

x = np.linspace(0, 100, 200)
plt.plot(x, ginibre_pcf_fct(x), 'b.')
plt.show()

Ogata

In [ ]:

k_norm2 = np.linspace(0.3, 30, 1000) # vector of wave length
k_norm2, sf_Ogata = sf_poisson_ball.hankel_quadrature(poisson_pcf_fct, method="Ogata", k_norm=k_norm, step_size=0.01,
                                                    nb_points=1000)

fig, axis = plt.subplots(figsize=(7,6))
fig = sf_poisson_ball.plot_isotropic_estimator(k_norm2, sf_Ogata, axis=axis,
                                               error_bar=True, bins=20,
                                                file_name="poisson_s_ho.pdf"
                                               )

Baddour-Chouinard

In [ ]:
k_norm3 = np.linspace(0.3, 30, 1000)
k_norm3, s_hbc = sf_poisson_ball.hankel_quadrature(poisson_pcf_fct,
                                                  k_norm=k_norm3,
                                                   r_max=r_max, 
                                                   nb_points=1000
                                                   )
fig, axis= plt.subplots(figsize=(7,6))
sf_poisson_ball.plot_isotropic_estimator(k_norm3, s_hbc, error_bar=True, bins=20, axis=axis, label=r"$\widehat{S}$", 
                                         file_name="poisson_s_hbc.pdf"
                                           )

In [ ]:
# save results 
import pickle
my_dict2 = {"poisso,_s_bi": [k_norm_allowed, s_bi], "poisson_s_ho": [k_norm2, s_ho], "poisson_s_hbc":[k_norm3, s_hbc] }
with open('poisson_s_ise.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

In [ ]:
# try to amelioreate ogata
from structure_factor.point_process import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
# Poisson points list
poisson_intensity=1/np.pi
poisson = HomogeneousPoissonPointProcess(intensity=poisson_intensity)

# BallWindow
window_ball = BallWindow(center=[0, 0], radius=300)
poisson_ball_big = poisson.generate_sample(window=window_ball)


# PointPattern
poisson_pp_big = PointPattern(poisson_ball_big, window_ball, poisson_intensity)
poisson_pp_big.plot()

In [ ]:
from structure_factor.structure_factor import StructureFactor
sf_poisson_big = StructureFactor(poisson_pp_big)

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

poisson_pcf_fv = pcf.estimate(poisson_pp_big, method="fv", Kest=dict(rmax=200), fv=dict(method="b", spar=0.3))
pcf.plot(poisson_pcf_fv, figsize=(7,6),
         color=['grey'], style=["."],
         file_name="pcf_fv_poisson.png")

r=poisson_pcf_fv["r"]
pcf_r = poisson_pcf_fv["pcf"]
poisson_pcf_fct = pcf.interpolate(r=r, pcf_r=pcf_r, clean=True, drop=True)
r_max = np.max(r)

In [ ]:
from structure_factor.statistics import apply_estimator
from multiprocessing import Pool
from functools import partial
from structure_factor.statistics import pcf_interpolate_list


core_num=8

n_allowed_k_norm=64
k_norm_allowed = get_k_norm(poisson_pp_big, n_allowed_k_norm=n_allowed_k_norm)

In [ ]:
k_norm_allowed

In [ ]:

k_norm = np.linspace(0.01,10, 1000) # vector of wave length
k_norm, sf_Ogata = sf_poisson_big.hankel_quadrature(poisson_pcf_fct, method="Ogata", k_norm=k_norm, step_size=0.1,
                                                    nb_points=1000)

fig, axis = plt.subplots(figsize=(7,6))
fig = sf_poisson_big.plot_isotropic_estimator(k_norm, sf_Ogata, axis=axis,
                                               error_bar=True, bins=60,
                                                file_name="poisson_s_ho.png"
                                               )

## 2.3- Hyperuniformity

In [ ]:
from structure_factor.hyperuniformity import Hyperuniformity
hyperuniformity_test = Hyperuniformity(k_norm, s_bi)
bin_center, bin_mean, bin_std = hyperuniformity_test.bin_data(bins=40)

### 2.3.1- Effective hyperuniformity

In [ ]:
H_ginibre, std = hyperuniformity_test.effective_hyperuniformity(k_norm_stop=2)
print("H_ginibre=", H_ginibre)

In [ ]:
#k_norm = hyperuniformity_test.k_norm
#sf = hyperuniformity_test.sf
sf = s_bi
fitted_sf_line = hyperuniformity_test.fitted_line # fitted line to sf
index_peak = hyperuniformity_test.i_first_peak # index of first peak of sf
x = np.linspace(0, 5, 300)
y = np.linspace(0,np.max(k_norm), 500)
fig=plt.figure(figsize=(10,6))
plt.plot(k_norm, sf, 'b.', label="approx_sf")
plt.plot(k_norm, sf, 'b', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.plot(y, utils.structure_factor_ginibre(y), 'g', label="exact sf")
#plt.plot(mean_k_norm[index_peak], mean_sf[index_peak], 'k*', label="first peak")
plt.legend()
plt.xlabel('wavelength ($||\mathbf{k}||$)')
plt.ylabel('Structure factor ($\mathsf{S}(\mathbf{k})$)')
plt.show()
#fig.savefig("fitted_si_ginibre.pdf", bbox_inches="tight")

### 2.3.2- Hyperuniformity class

In [ ]:
sf_power_decay, c = hyperuniformity_test.hyperuniformity_class(k_norm_stop=0.5)
print(sf_power_decay)


In [ ]:

#k_norm = hyperuniformity_test.k_norm
#sf = hyperuniformity_test.sf
sf = s_bi
fitted_poly = hyperuniformity_test.fitted_poly
x = np.linspace(0, 2, 300)
y = np.linspace(0,np.max(k_norm), 500)
fig=plt.figure(figsize=(10,6))
plt.plot(k_norm, sf, 'b.', label="approx_sf")
plt.plot(k_norm, sf, 'b.', label="approx_sf")

plt.plot(k_norm, sf, 'b', label="approx_sf")
plt.plot(y, utils.structure_factor_ginibre(y), 'g', label="exact sf")
plt.plot(x, fitted_poly(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength ($||\mathbf{k}||$)')
plt.ylabel('Structure factor ($\mathsf{S}(\mathbf{k})$)')
plt.show()
#fig.savefig("fitted_si_ginibre.pdf", bbox_inches="tight")

## 2.4- Statistics

In [ ]:
# load points
import pickle
path = "/Users/dhawat/github/structure_factor_paper/test_data/list_samples_poisson_pp.pkl"
with open(path, "rb") as input_file:
    list_sample_poisson_pp = pickle.load(input_file)
    
from structure_factor.spatial_windows import BallWindow, BoxWindow
poisson_ball_pp_list = list_sample_poisson_pp["poisson_pp_ball"]
poisson_ball_pp_list[0].plot(window_res=BallWindow(radius=50, center=(0,0)) )

poisson_box_pp_list = list_sample_poisson_pp["poisson_pp_box"]
poisson_box_pp_list[0].plot()

In [ ]:
# statistics
from structure_factor.statistics import SummaryStatistics, get_k, get_k_norm

poisson_ball_statistics = SummaryStatistics(poisson_ball_pp_list)
poisson_box_statistics = SummaryStatistics(poisson_box_pp_list)

### 2.4.2- For DSE

In [ ]:
from structure_factor.statistics import apply_estimator
from multiprocessing import Pool
from functools import partial
from structure_factor.tapers import SineTaper
import time

start_time = time.time()
core_num=8

k_allowed = get_k(poisson_box_pp_list[0], k_max=2)
x = np.linspace(np.min(np.abs(k_allowed)), np.max(np.abs(k_allowed)), 30)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
p = [1, 1]
taper = SineTaper(p)

if __name__ == "__main__":
    # scattering intensity
    s_si_allowed_k = poisson_box_statistics.sample_approximation(estimator="scattering_intensity", k_max=2)
    # scattering intensity debiased
    s_dtp_t0 = poisson_box_statistics.sample_approximation(estimator="scattering_intensity", k=k, debiased=True, direct=True)
    # debiased tapered periodogram (sinusoidal taper)
    s_dtp_t1 = poisson_box_statistics.sample_approximation(estimator="tapered_periodogram", k=k,taper=taper, debiased=True, direct=True)
    # multitapered periodogram (sinusoidal taper)
    s_mdtp = poisson_box_statistics.sample_approximation(estimator="multitapered_periodogram", k=k, debiased=True, direct=True)
    
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
norm_k_allowed1 = utils.norm_k(k_allowed)
norm_k = utils.norm_k(k)
print(np.max(norm_k_allowed1), np.max(norm_k), np.min(norm_k_allowed1), np.min(norm_k))

In [ ]:
from structure_factor.statistics import sample_statistics
k_norm_si, m_si, ivar_si, ibias_si, imse_si, imse_var_si = sample_statistics(k=k_allowed,
                                                                                approximation=s_si_allowed_k,
                                                                                exact=utils.structure_factor_poisson )
k_norm_dtp_t0, m_dtp_t0, ivar_dtp_t0, ibias_dtp_t0, imse_dtp_t0, imse_var_dtp_t0 = sample_statistics(
                                                                                k=k,
                                                                                approximation=s_dtp_t0,
                                                                                exact=utils.structure_factor_poisson )
k_norm_dtp_t1, m_dtp_t1, ivar_dtp_t1, ibias_dtp_t1, imse_dtp_t1, imse_var_dtp_t1 = sample_statistics(k=k,
                                                                                approximation=s_dtp_t1,
                                                                                exact=utils.structure_factor_poisson )
k_norm_dtp, m_mdtp, ivar_mdtp, ibias_mdtp, imse_mdtp, imse_var_mdtp = sample_statistics(k=k,
                                                                                approximation=s_mdtp,
                                                                                exact=utils.structure_factor_poisson )
print("For s_si: ivar", ivar_si, "ibias", ibias_si, "imse", imse_si, "3std(imse)",  np.sqrt(imse_var_si)*3)
print("For s_dtp_t0: ivar", ivar_dtp_t0, "ibias", ibias_dtp_t0, "imse", imse_dtp_t0, "3std(imse)",  np.sqrt(imse_var_dtp_t0)*3)
print("For s_dtp_t1: ivar", ivar_dtp_t1, "ibias", ibias_dtp_t1, "imse", imse_dtp_t1, "3std(imse)",  np.sqrt(imse_var_dtp_t1)*3)
print("For s_mdtp: ivar", ivar_mdtp, "ibias", ibias_mdtp, "imse", imse_mdtp, "3std(imse)",  np.sqrt(imse_var_mdtp)*3)

In [ ]:
norm_k_allowed1 = utils.norm_k(k_allowed)
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(7,6))

axis.loglog(k_norm_si, m_si, 'b^', label=r"$\overline{\widehat{S}_{\mathrm{SI}}(2\pi \mathbf{n}/L)}$")
#axis.errorbar(k_norm_si, m_si, yerr=var_si, lw=1, ecolor='b', capsize=3, capthick=1)
axis.loglog(k_norm_dtp_t0, m_dtp_t0, 'k*', label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_0, \mathbf{k})}$")
#axis.errorbar(k_norm_dtp_t0, m_dtp_t0, yerr=var_dtp_t0, lw=1, ecolor='k', capsize=3, capthick=1)
axis.loglog(k_norm_dtp_t1, m_dtp_t1, 'm+',label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_1, \mathbf{k})}$")
#axis.errorbar(k_norm_dtp_t1, m_dtp_t1, yerr=var_dtp_t1, lw=1, ecolor='m', capsize=3, capthick=1)
axis.loglog(k_norm_mdtp, m_mdtp, 'c.',  label=r"$\overline{\widehat{S}_{\mathrm{DDMTP}}((t_j)_{j=1}^4, \mathbf{k})}$")
utils.plot_exact(norm_k_allowed1, utils.structure_factor_poisson(norm_k_allowed1), axis=axis, label=r"$S(\mathbf{k})$")
axis.legend()
axis.set_xlabel(r"Wavenumber ($k$)")
axis.set_ylabel(r"Structure factor ($S(k)$)")
fig.savefig("comparaison_DSE_mean_poisson.pdf", bbox_inches="tight")

In [ ]:
# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=16)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

In [ ]:
norm_k_allowed1 = utils.norm_k(k_allowed)
fig, axis = plt.subplots(figsize=(7,6))
utils.plot_summary_2(norm_k_allowed1,m_si, c='b',marker='*', bins=30, axis=axis, label=r"$\overline{\widehat{S}_{\mathrm{SI}}(2\pi \mathbf{n}/L)}$")
utils.plot_summary_2(norm_k_allowed1,m_dtp_t0, c='k',marker='.', bins=30, axis=axis, label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_0, \mathbf{k})}$")
utils.plot_summary_2(norm_k_allowed1,m_dtp_t1, c='m',marker='+', bins=30, axis=axis, label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_1, \mathbf{k})}$")
utils.plot_summary_2(norm_k_allowed1,m_mdtp, c='grey',marker='o', bins=30, axis=axis, label=r"$\overline{\widehat{S}_{\mathrm{MDDTP}}((t_j)_{j=1}^4, \mathbf{k})}$")
axis.loglog(norm_k_allowed1, utils.structure_factor_poisson(norm_k_allowed1), 'g', label=r"$S(\mathbf{k})$")
axis.legend(loc=4)
axis.set_xlabel(r"Wavenumber ($k$)")
axis.set_ylabel(r"Structure factor ($S(k)$)")
fig.savefig("comparaison_DSE_mean_poisson.png", bbox_inches="tight")

### 2.4.2- For ISE

In [ ]:
r_max

In [ ]:
from structure_factor.statistics import apply_estimator
from multiprocessing import Pool
from functools import partial
from structure_factor.statistics import pcf_interpolate_list

start_time = time.time()
core_num=8

n_allowed_k_norm=64
k_norm_allowed = get_k_norm(poisson_ball_pp_list[0], n_allowed_k_norm=n_allowed_k_norm)

if __name__ == "__main__":
    # Bartlett's isotropic estimator
    s_bi = poisson_ball_statistics.sample_approximation(estimator="bartlett_isotropic_estimator", n_allowed_k_norm=n_allowed_k_norm)
    # list pcf
    pcf_list = poisson_ball_statistics.sample_pcf_approximation(method="fv", Kest=dict(rmax=r_max), fv=dict(method="b", spar=0.1))
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
s = len(pcf_list)
# interpolate pcf
r_list = [pcf_list[i]["r"] for i in range(s)]
pcf_fv_list = [pcf_list[i]["pcf"] for i in range(s)]
pcf_fct_list = pcf_interpolate_list(r_list, pcf_fv_list, drop=True)
# BaddourChouinard
s_hbc_list = poisson_ball_statistics.sample_integral_approximation(pcf_fct_list,method="BaddourChouinard", 
                                                                   k_norm=k_norm_allowed, r_max=30,  nb_points=1000) 
# Ogata
s_ho_list = poisson_ball_statistics.sample_integral_approximation(pcf_fct_list,method="Ogata", 
                                                                   k_norm=k_norm_allowed, step_size=0.01,  nb_points=500) 


In [ ]:
# save results 
import pickle
my_dict2 = {"poisson_s_bi_list": [k_norm_allowed, s_bi], "poisson_s_ho_list": [k_norm_allowed, s_ho_list],
            "poisson_s_hbc_list": [k_norm_allowed, s_hbc_list]}
with open('poisson_ise_list.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

In [ ]:



k_bi, m_bi, ivar_bi, ibias_bi, imse_bi, imse_var_bi = sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_bi,
                                                                                exact=utils.structure_factor_poisson)
k_hbc, m_hbc, ivar_hbc, ibias_hbc, imse_hbc, imse_var_hbc =sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_hbc_list,
                                                                                exact=utils.structure_factor_poisson )
k_ho, m_ho, ivar_ho, ibias_ho, imse_ho, imse_var_ho = sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_ho_list,
                                                                                exact=utils.structure_factor_poisson )
print("For s_bi: ivar=", ivar_bi, "ibias=", ibias_bi, "imse=", imse_bi, "3std(imse)",  np.sqrt(imse_var_bi)*3)

print("For s_hbc: ivar=", ivar_hbc, "ibias=", ibias_hbc, "imse=", imse_hbc, "3std(imse)",  np.sqrt(imse_var_hbc)*3)

print("For s_ho: ivar=", ivar_ho, "ibias=", ibias_ho, "imse=", imse_ho, "3std(imse)",  np.sqrt(imse_var_ho)*3)

In [ ]:
ivar_ho + ibias_ho

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
axis.loglog(k_norm_allowed, m_bi, 'm*', label=r"$\overline{\widehat{S}_{\mathrm{BI}}(k)}$")
#axis.loglog(k_norm_allowed, m_bi, 'm')
axis.loglog(k_ho, m_ho, 'b.', label=r"$\overline{\widehat{S}_{\mathrm{HO}}(k)}$")
#axis.loglog(k_norm_allowed, m_ho, 'b')
axis.loglog(k_hbc, m_hbc, 'k+', label=r"$\overline{\widehat{S}_{\mathrm{HBC}}(k)}$")
#axis.loglog(k_norm_allowed, m_hbc, 'k')
axis.loglog(k_norm_allowed, utils.structure_factor_poisson(k_norm_allowed), 'g', label= r"$S(k)$")
axis.legend()
axis.set_xlabel(r"Wavenumber ($k$)")
axis.set_ylabel(r"Structure factor ($S(k)$)")
plt.show()
fig.savefig("comparaison_ISE_mean_poisson.pdf", bbox_inches="tight")

## 2.4.3- Hyperuniformity study

In [ ]:
from structure_factor.statistics import apply_estimator
from multiprocessing import Pool
from functools import partial
import time

start_time = time.time()
core_num=8

n_allowed_k_norm=64
k_norm_allowed = get_k_norm(poisson_ball_pp_list[0], n_allowed_k_norm=n_allowed_k_norm)

if __name__ == "__main__":
    # Bartlett's isotropic estimator
    s_bi = poisson_ball_statistics.sample_approximation(estimator="bartlett_isotropic_estimator", n_allowed_k_norm=n_allowed_k_norm)
    
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# save sample s_bi:
import pickle
my_dict = {"k_norm": k_norm_allowed, "list_s_bi_poisson": s_bi}
with open('sample_s_bi_poisson.pkl', 'wb') as f:
       pickle.dump(my_dict, f)

### 2.4.3.1- Effective Hyperuniformity 

In [ ]:
from structure_factor.statistics import sample_hyperuniformity, sample_variance

H_index_list = sample_hyperuniformity(k_norm_allowed, s_bi, estimator="effective", k_norm_stop=1)
H_var, H_m = sample_variance(H_index_list)
print("mean H index", H_m, "3std H index", 3*np.sqrt(H_var))

# 3- KLY process

In [ ]:
import pickle
# load points
path = "/Users/dhawat/github/structure_factor_paper/test_data/list_samples_kly_pp.pkl"
with open(path, "rb") as input_file:
    list_sample_kly_pp = pickle.load(input_file)

In [ ]:
from structure_factor.spatial_windows import BallWindow, BoxWindow
list_kly_pp_ball = list_sample_kly_pp["kly_pp_ball"]
kly_pp_ball = list_kly_pp_ball[2]
kly_pp_ball.plot(window_res=BallWindow(radius=20, center=(0,0)), file_name="kly_pp_ball.pdf" )

list_kly_pp_box = list_sample_kly_pp["kly_pp_box"]
kly_pp_box = list_kly_pp_box[2]
kly_pp_box.plot(file_name="kly_pp_box.pdf", window_res=BoxWindow(bounds=[[-20, 20], [-20, 20]]))

## 3.1- Direct spectral estimator

In [ ]:
sf_kly_box = StructureFactor(kly_pp_box)

### 3.1.1- Scattering intensity

In [ ]:
# on allowed values 
k_max = 15
allowed_k, si = sf_kly_box.scattering_intensity(k_max=k_max)
print("Scattering intensity")



In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_kly_box.plot_spectral_estimator(allowed_k, si, axes=axis, plot_type="radial",
                                           label=r"$\widehat{S}$",
                                            bins=80, error_bar=True, 
                                           file_name="si_allowed_k_kly.pdf"
                                          )

In [ ]:
x = np.linspace(0, 15, 200)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# on k 
k, si1 = sf_kly_box.scattering_intensity(k=k, debiased=False)
# on k debiased directly
k, si2 = sf_kly_box.scattering_intensity(k=k, debiased=True)
# on k debiased undirectly
k, si3 = sf_kly_box.scattering_intensity(k=k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_kly_box.plot_spectral_estimator(k, si3, axes=axis, positive=False, plot_type="radial",
                                           label=r"$\widehat{S}$",
                                            bins=60, error_bar=True, 
                                         file_name="si_uddebiased_kly.pdf")

In [ ]:
# save results 
import pickle
my_dict2 = {"kly_s_si_allowed_k": [allowed_k, si], "kly_s_si_k": [k, si1], "kly_s_si_directly_debiased": [k, si2],
            "kly_s_si_undirectly_debiased": [k, si3]}
with open('kly_s_si.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

### 3.1.2- Tapered periodogram with SineTaper

In [ ]:
from structure_factor.tapers import SineTaper
## tapered periodogram
x = np.linspace(0, k_max, 200)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
p = [1, 1]
taper = SineTaper(p)
k, s_tp = sf_kly_box.tapered_periodogram(k, taper, debiased=False)
k, s_dtp = sf_kly_box.tapered_periodogram(k, taper)
k, s_udtp = sf_kly_box.tapered_periodogram(k, taper, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_kly_box.plot_spectral_estimator(k, s_tp, axes=axis, positive=False, plot_type="radial",
                                            bins=100, error_bar=True, label=r"$\widehat{S}$",
                                           file_name="s_tp_sine_taper_kly.pdf"
                                          )

In [ ]:
# save results 
import pickle
my_dict2 = {"kly_s_tp_sine_taper": [k, s_tp], "kly_s_ddtp_sine_taper": [k, s_dtp], "kly_s_udtp_sine_taper": [k, s_udtp]}
with open('kly_s_tp_sine_taper.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

### 3.1.3- Multitapering

In [ ]:
from structure_factor.tapers import SineTaper
## multitapered tapered periodogram
x = np.linspace(0, k_max, 200)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
k, s_mtp = sf_kly_box.multitapered_periodogram(k, debiased=False)
k, s_ddmtp = sf_kly_box.multitapered_periodogram(k)
k, s_udmtp = sf_kly_box.multitapered_periodogram(k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_kly_box.plot_spectral_estimator(k, s_mtp, axes=axis, positive=False, plot_type="radial",
                                            bins=80, error_bar=True, label=r"$\widehat{S}$",
                                            file_name="s_mtp_sine_taper_kly.pdf"
                                          )

In [ ]:
# save results 
import pickle
my_dict2 = {"kly_s_mtp_sine_taper": [k, s_mtp], "kly_s_ddmtp_sine_taper": [k,s_ddmtp], "kly_s_udmtp_sine_taper": [k, s_udmtp]}
with open('kly_s_mtp_sine_taper.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

## 3.2- Isotropic estimators

In [ ]:
sf_kly_ball = StructureFactor(kly_pp_ball)

### 3.2.1- Bartlett isotropic estimators

In [ ]:
window_res = BallWindow(center=[0, 0], radius=70)
sf_kly_ball = StructureFactor(kly_pp_ball.restrict_to_window(window_res))

In [ ]:
import time
start_time = time.time()
k_norm_allowed, s_bi = sf_kly_ball.bartlett_isotropic_estimator(n_allowed_k_norm=150)
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
k_norm_allowed

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
sf_kly_ball.plot_isotropic_estimator(k_norm_allowed, s_bi, axis=axis,  label=r"$\widehat{S}$",
                                     file_name="s_bi_kly.pdf")

In [ ]:
k_non_allowed = np.linspace(min(k_norm_allowed)+ 0.003, max(k_norm_allowed), 150)
k_norm_non_allowed, s_bi_non_allowed = sf_kly_ball.bartlett_isotropic_estimator(k_non_allowed)
fig, axis = plt.subplots(figsize=(7,6))
sf_kly_ball.plot_isotropic_estimator(k_norm_non_allowed, s_bi_non_allowed, axis=axis,
                                    label=r"$\widehat{S}$", 
                                     file_name="s_bi_kly_non_allowed.pdf")

### 3.2.2- Pair correlation function

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf


pcf.ppp

In [ ]:
r= np.linspace(0, 20, 200)
kly_pcf_ppp = pcf.estimate(kly_pp_ball, method="ppp", r=r, correction="all")
pcf.plot(kly_pcf_ppp, figsize=(7,6),
         color=['grey', 'b', 'darkcyan'], style=[".", "*", "^"],
         file_name="pcf_ppp_kly.pdf")

pcf.fv

In [ ]:
kly_pcf_fv = pcf.estimate(kly_pp_ball, method="fv", Kest=dict(rmax=20), fv=dict(method="b", spar=0.2))
pcf.plot(kly_pcf_fv, figsize=(7,6),
         color=['grey'], style=["."],
         file_name="pcf_fv_kly.pdf")

Interpolation

In [ ]:
r = kly_pcf_fv["r"]
r_max = np.max(r)
pcf_r = kly_pcf_fv["pcf"]
kly_pcf_fct = pcf.interpolate(r=r, pcf_r=pcf_r, drop=True)

x = np.linspace(0, 100, 200)
plt.plot(x, kly_pcf_fct(x), 'b.')


### 3.2.3- Hankel transform

Ogata

In [ ]:
k_norm2 = np.linspace(0.3, 30, 1000) # vector of wave length
k_norm2, s_ho = sf_kly_ball.hankel_quadrature(kly_pcf_fct, method="Ogata", k_norm=k_norm2, step_size=0.01,
                                                    nb_points=500, r_max=r_max)


In [ ]:
fig, axis= plt.subplots(figsize=(7,6))
fig = sf_kly_ball.plot_isotropic_estimator(k_norm2, s_ho, axis=axis, 
                                           label=r"$\widehat{S}$",  
                                           #error_bar=True, bins=40, 
                                            file_name="kly_s_ho.pdf"
                                           )

Baddour-Chouinard

In [ ]:
k_norm3 = np.linspace(0.3, 30, 1000)
k_norm3, s_hbc = sf_kly_ball.hankel_quadrature(kly_pcf_fct, k_norm=k_norm3, r_max=r_max, nb_points=3000)

In [ ]:
fig, axis= plt.subplots(figsize=(7,6))
sf_kly_ball.plot_isotropic_estimator(k_norm3, s_hbc, axis=axis, error_bar=True, bins=100, 
                                     label=r"$\widehat{S}$" , file_name="kly_s_hbc.pdf")

In [ ]:
# save results 
import pickle
my_dict2 = {"kly_s_bi": [k_norm_allowed, s_bi], "kly_s_ho": [k_norm2, s_ho], "kly_s_hbc":[k_norm3, s_hbc] }
with open('kly_s_ise.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

## 3.3- Hyperuniformity

In [ ]:
from structure_factor.hyperuniformity import Hyperuniformity
hyperuniformity_test = Hyperuniformity(k_norm, s_bi)
#bin_center, bin_mean, bin_std = hyperuniformity_test.bin_data(bins=80)

### 3.3.1- Effective hyperuniformity

In [ ]:
H_poisson, std = hyperuniformity_test.effective_hyperuniformity(k_norm_stop=1)
print("H_poisson=", H_poisson)

In [ ]:
#k_norm = hyperuniformity_test.k_norm
#sf = hyperuniformity_test.sf
sf = s_bi
fitted_sf_line = hyperuniformity_test.fitted_line # fitted line to sf
index_peak = hyperuniformity_test.i_first_peak # index of first peak of sf
x = np.linspace(0, 5, 300)
fig=plt.figure(figsize=(10,6))
plt.plot(k_norm, sf, 'b.', label="approx_sf")
plt.plot(k_norm, sf, 'b', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
#plt.plot(mean_k_norm[index_peak], mean_sf[index_peak], 'k*', label="first peak")
plt.legend()
plt.xlabel('wavelength ($||\mathbf{k}||$)')
plt.ylabel('Structure factor ($\mathsf{S}(\mathbf{k})$)')
plt.show()
#fig.savefig("fitted_si_ginibre.pdf", bbox_inches="tight")

### 3.3.2- Hyperuniformity class


In [ ]:
sf_power_decay, c = hyperuniformity_test.hyperuniformity_class(k_norm_stop=0.9)
print(sf_power_decay)

In [ ]:
#k_norm = hyperuniformity_test.k_norm
#sf = hyperuniformity_test.sf
sf = s_bi
fitted_poly = hyperuniformity_test.fitted_poly
x = np.linspace(0, 2, 300)
fig=plt.figure(figsize=(10,6))
plt.plot(k_norm[:40], sf[:40], 'b.', label="approx_sf")
#plt.plot(k_norm, sf, 'b', label="approx_sf")
plt.plot(x, fitted_poly(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength ($||\mathbf{k}||$)')
plt.ylabel('Structure factor ($\mathsf{S}(\mathbf{k})$)')
plt.show()
#fig.savefig("fitted_si_ginibre.pdf", bbox_inches="tight")

## 3.4- Statistics

In [ ]:
# Generate 50 samples from KLY process 
import rpy2.robjects as robjects
from spatstat_interface.utils import install_r_package
import numpy as np 

r = robjects.r
r.source('MatchingPP.R')


# Install_r_package("RANN")
list_KLY_points = []
# 50 KLY sample
for i in range(50):
    print(i)
    my_dict = {"dimension": 2, "linear.system.size":210.0, "intensity":11.0, "verbose": False}
    data_r = r.MatchingPoissonToLattice(**my_dict)
    KLY_process = np.array(data_r)
    KLY_process = np.column_stack((KLY_process[:, 0], KLY_process[:, 1])) - 105
    list_KLY_points.append(KLY_process)

In [ ]:
from structure_factor.point_pattern import pointpattern_list
from structure_factor.spatial_windows import BallWindow, BoxWindow

s=50 # number of samples
kly_intensity=1
window_big = BoxWindow([[-105, 105], [-105, 105]])

# KLY pp ball
R = 100
window_ball = BallWindow(center=[0,0], radius=R)
kly_ball_pp_list = pointpattern_list(list_points=list_KLY_points, window=window_big, intensity=kly_intensity,
                                        window_res=window_ball)
kly_ball_pp_list[0].plot()

# KLY pp box
l_box = 70
window_box = BoxWindow(bounds=[[-l_box/2, l_box/2], [-l_box/2, l_box/2]])
kly_box_pp_list = pointpattern_list(list_points=list_KLY_points, window=window_big, intensity=kly_intensity,
                                        window_res=window_box)
kly_box_pp_list[0].plot()


In [ ]:
# save the samples
import pickle
my_dict2 = {"kly_pp_ball": kly_ball_pp_list, "kly_pp_box": kly_box_pp_list}
with open('samples_kly_pp.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

Initialize the class statistics

In [ ]:
# Initialize statistics
from structure_factor.statistics import SummaryStatistics, get_k, get_k_norm

kly_ball_statistics = SummaryStatistics(kly_ball_pp_list)
kly_box_statistics = SummaryStatistics(kly_box_pp_list)

### 3.4.1- Hyeruniformity study

In [ ]:
import time

start_time = time.time()
core_num=7

n_allowed_k_norm=32
k_norm_allowed = get_k_norm(kly_ball_pp_list[0], n_allowed_k_norm=n_allowed_k_norm)

if __name__ == "__main__":
    # Bartlett's isotropic estimator
    s_bi = kly_ball_statistics.sample_approximation(estimator="bartlett_isotropic_estimator", n_allowed_k_norm=n_allowed_k_norm)
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# save sample s_bi:
import pickle
my_dict = {"k_norm": k_norm_allowed, "list_s_bi_kly": s_bi}
with open('sample_s_bi_kly.pkl', 'wb') as f:
       pickle.dump(my_dict, f)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(k_norm_allowed, s_bi[0], 'b.')

In [ ]:
### 3.4.1- Effective hyperuniformity

In [ ]:
from structure_factor.statistics import sample_hyperuniformity, sample_variance

H_index_list = sample_hyperuniformity(k_norm_allowed, s_bi, estimator="effective", k_norm_stop=1)
H_var, H_m = sample_variance(H_index_list)
print("mean H index", H_m, "3std H index", 3*np.sqrt(H_var))

In [ ]:
### 3.4.2- Hyperuniformity class

alpha_list = sample_hyperuniformity(k_norm_allowed, s_bi, estimator="class", k_norm_stop=0.45)
alpha_var, alpha_m = sample_variance(alpha_list)
print("mean power decay", alpha_m, "3std power decay", 3*np.sqrt(alpha_var))


# 4- Thomas process

In [ ]:
from structure_factor.point_process import ThomasProintProcess

In [ ]:
# load points
path = "/Users/dhawat/github/structure_factor_paper/test_data/list_samples_thomas_pp.pkl"
with open(path, "rb") as input_file:
    list_sample_thomas_pp = pickle.load(input_file)

In [ ]:
list_sample_thomas_pp.keys()

In [ ]:
from structure_factor.spatial_windows import BallWindow, BoxWindow
list_thomas_pp_ball = list_sample_thomas_pp["list_samples_thomas_pp_ball"]
thomas_pp_ball = list_thomas_pp_ball[1]
thomas_pp_ball.plot( file_name="thomas_pp_ball.pdf", window_res=BallWindow(radius=50, center=[0,0]) )

list_thomas_pp_box = list_sample_thomas_pp["list_samples_thomas_pp_box"]
thomas_pp_box = list_thomas_pp_box[1]
thomas_pp_box.plot(file_name="thomas_pp_box.pdf")

## 4.1- Direct spectral estimator

In [ ]:
from structure_factor.structure_factor import StructureFactor
sf_thomas_box = StructureFactor(thomas_pp_box)

### 4.1.1- Scattering intensity

In [ ]:
# on allowed values 
k_max = 6
allowed_k, si = sf_thomas_box.scattering_intensity(k_max=k_max)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_thomas_box.plot_spectral_estimator(allowed_k, si, axes=axis, plot_type="radial",exact_sf=thomas.structure_factor,
                                            bins=40, error_bar=True, 
                                            label=r"$\widehat{S}$",
                                            file_name="si_allowed_k_thomas.pdf"
                                             )

In [ ]:
x = np.linspace(0, k_max, 150)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
# on k 
k, si1 = sf_thomas_box.scattering_intensity(k=k, debiased=False)
# on k debiased directly
k, si2 = sf_thomas_box.scattering_intensity(k=k, debiased=True)
# on k debiased undirectly
k, si3 = sf_thomas_box.scattering_intensity(k=k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_thomas_box.plot_spectral_estimator(k, si1, axes=axis, plot_type="radial",exact_sf=thomas.structure_factor,
                                            bins=60, error_bar=True, 
                                            label=r"$\widehat{S}$",
                                            file_name="si_k_thomas.pdf"
                                             )

In [ ]:
# save results 
import pickle
my_dict2 = {"thomas_s_si_allowed_k": [allowed_k, si], "thomas_s_si_k": [k, si1], "thomas_s_si_directly_debiased": [k, si2],
            "thomas_s_si_undirectly_debiased": [k, si3]}
with open('thomas_s_si.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

### 4.1.2- Tapered periodogram SineTaper

In [ ]:
from structure_factor.tapers import SineTaper
## tapered periodogram
x = np.linspace(0, k_max, 150)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
p = [1, 1]
taper = SineTaper(p)
_, s_tp = sf_thomas_box.tapered_periodogram(k, taper, debiased=False)
_, s_dtp = sf_thomas_box.tapered_periodogram(k, taper)
_, s_udtp = sf_thomas_box.tapered_periodogram(k, taper, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_thomas_box.plot_spectral_estimator(k, s_udtp, axes=axis, positive=False, plot_type="radial", exact_sf=thomas.structure_factor,
                                            bins=60, error_bar=True, label=r"$\widehat{S}$",
                                           file_name="s_udtp_sine_taper_thomas.pdf"
                                          )

In [ ]:
# save results 
import pickle
my_dict2 = {"thomas_s_tp_sine_taper": [k, s_tp], "thomas_s_ddtp_sine_taper": [k, s_dtp], "thomas_s_udtp_sine_taper": [k, s_udtp]}
with open('thomas_s_tp_sine_taper.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

### 4.1.3- Multitapering

In [ ]:
from structure_factor.tapers import SineTaper
## multitapered tapered periodogram
x = np.linspace(0, 6, 150)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
_, s_mtp = sf_thomas_box.multitapered_periodogram(k, debiased=False)
_,s_ddmtp = sf_thomas_box.multitapered_periodogram(k, debiased=True, direct=True) #or simply sf_thomas_res.multitapered_periodogram(k)
_,s_udmtp = sf_thomas_box.multitapered_periodogram(k, debiased=True, direct=False)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
fig = sf_thomas_box.plot_spectral_estimator(k, s_mtp, axes=axis, positive=False, plot_type="radial",exact_sf=thomas.structure_factor,
                                            bins=80, error_bar=True, label=r"$\widehat{S}$",
                                            file_name="s_mtp_sine_taper_thomas.pdf"
                                          )

In [ ]:
# save results 
import pickle
my_dict2 = {"thomas_s_mtp_sine_taper": [k, s_mtp], "thomas_s_ddmtp_sine_taper": [k,s_ddmtp], "thomas_s_udmtp_sine_taper": [k, s_udmtp]}
with open('thomas_s_mtp_sine_taper.pkl', 'wb') as f:
       pickle.dump(my_dict2, f)

## 4.2- Isotropic estimators

In [ ]:
sf_thomas_ball = StructureFactor(thomas_pp_ball)

### 4.2.1- Bartlett isotropic estimator

Allowed wavenumbers

In [ ]:
import time
time_start = time.time
k_norm, s_bi = sf_thomas_ball.bartlett_isotropic_estimator(n_allowed_k_norm=100)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
sf_th_exact = lambda k: utils.structure_factor_thomas(k, lambdaDaughter, sigma )
sf_thomas_ball.plot_isotropic_estimator(k_norm, s_bi, axis=axis, exact_sf=thomas.structure_factor,
                                         label=r"$\widehat{S}$", file_name="s_bi_thomas.pdf")

Non allowed wavenumbers

In [ ]:
k = np.linspace(min(k_norm) + 0.002, max(k_norm), 150)
k_norm_non_allowed, s_bi_nonallowed = sf_thomas_ball.bartlett_isotropic_estimator(k_norm=k)

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
sf_th_exact = lambda k: utils.structure_factor_thomas(k, lambdaDaughter, sigma )
sf_thomas_ball.plot_isotropic_estimator(k_norm_non_allowed, s_bi_nonallowed, axis=axis, exact_sf=thomas.structure_factor,
                                         label=r"$\widehat{S}$", file_name="s_bi_thomas_non_allowed_k.pdf")

### 4.2.2- Pair correlation function

In [ ]:
from structure_factor.pair_correlation_function import PairCorrelationFunction as pcf

In [ ]:
ext_pcf_thomas = lambda r: utils.pair_correlation_function_thomas(r, d=2, rho_parent=lambdaParent, sigma=sigma)

pcf.ppp

In [ ]:
r= np.linspace(0, 20, 200)
thomas_pcf_ppp = pcf.estimate(thomas_pp_ball, method="ppp", r=r, correction="all")


In [ ]:
pcf.plot(thomas_pcf_ppp, figsize=(7,6),
         exact_pcf=thomas.pair_correlation_function,
         color=['grey', 'b', 'darkcyan'], style=[".", "*", "^"],
         file_name="pcf_ppp_thomas.pdf")

pcf.fv

In [ ]:
thomas_pcf_fv = pcf.estimate(thomas_pp_ball, method="fv", Kest=dict(rmax=20), fv=dict(method="b", spar=0.2))


thomas_pcf_fv

In [ ]:
pcf.plot(thomas_pcf_fv, figsize=(7,6),
         exact_pcf=thomas.pair_correlation_function,
         color=['grey'], style=["."],
         file_name="pcf_fv_thomas.pdf"
        )

Interpolation

In [ ]:
thomas_pcf_fv

In [ ]:
r = thomas_pcf_fv["r"]
r_max = np.max(r)
pcf_r = thomas_pcf_fv["pcf"]
thomas_pcf_fct = pcf.interpolate(r=r, pcf_r=pcf_r, drop=True)

x = np.linspace(0, 100, 200)
plt.plot(x, thomas_pcf_fct(x), 'b.')


In [ ]:
r = np.linspace(0, 20, 200 )
plt.plot(r, thomas_pcf_fct(r), 'b')
plt.plot(r, thomas.pair_correlation_function(r), 'g')

#### 4.2.3- Hankel transform 

Ogata

In [ ]:
k_norm2 = np.linspace(0.3, 30, 3000) # vector of wave length
k_norm2, s_ho = sf_thomas_ball.hankel_quadrature(thomas_pcf_fct, method="Ogata", k_norm=k_norm2, step_size=0.01,
                                                    nb_points=1000)


In [ ]:
fig, axis =  plt.subplots(figsize=(7,6))
sf_thomas_ball.plot_isotropic_estimator(k_norm2, s_ho, axis=axis, exact_sf=thomas.structure_factor,
                                        #bins=100, error_bar=True, 
                                        label=r"$\widehat{S}$",
                                        file_name="thomas_s_ho.pdf"
                                           )

Baddour chouinard

In [ ]:
k_norm3 = np.linspace(0.3, 10, 400)
k_norm3, s_hbc = sf_thomas_ball.hankel_quadrature(thomas_pcf_fct, k_norm=k_norm3,  r_max=r_max, nb_points=3000)

In [ ]:
fig, axis= plt.subplots(figsize=(7,6))
sf_thomas_ball.plot_isotropic_estimator(k_norm3, s_hbc,axis=axis, exact_sf=thomas.structure_factor, 
                                        #bins=80, error_bar=True,
                                        label= r"$\widehat{S}$",file_name="thomas_s_hbc.pdf" )

## 4.3- Hyperuniformity

In [ ]:
from structure_factor.hyperuniformity import Hyperuniformity
hyperuniformity_test = Hyperuniformity(k_norm, s_bi)
bin_center, bin_mean, bin_std = hyperuniformity_test.bin_data(bins=50)

In [ ]:
bin_std

### 4.3.1- Effective hyperuniformity

In [ ]:
H_thomas, std = hyperuniformity_test.effective_hyperuniformity(k_norm_stop=0.5)
print("H_thomas=", H_thomas)

In [ ]:
sf_th_exact(0)

In [ ]:
#k_norm = hyperuniformity_test.k_norm
#sf = hyperuniformity_test.sf
sf = s_bi
fitted_sf_line = hyperuniformity_test.fitted_line # fitted line to sf
index_peak = hyperuniformity_test.i_first_peak # index of first peak of sf
x = np.linspace(0, 2, 300)
y = np.linspace(0,np.max(k_norm), 500)
fig=plt.figure(figsize=(10,6))
plt.plot(k_norm, sf, 'b.', label="approx_sf")
plt.plot(k_norm, sf, 'b', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.plot(y, sf_th_exact(y), 'g', label="exact sf")
#plt.plot(mean_k_norm[index_peak], mean_sf[index_peak], 'k*', label="first peak")
plt.legend()
plt.xlabel('wavelength ($||\mathbf{k}||$)')
plt.ylabel('Structure factor ($\mathsf{S}(\mathbf{k})$)')
plt.show()
#fig.savefig("fitted_si_ginibre.pdf", bbox_inches="tight")

### 4.3.2- Hyperuniformity class 

# NO NEED TO RUN THE TEST AS THE EFFECTIVE TEST SHOW NON HYPERUNIFORMITY

In [ ]:
sf_power_decay, c = hyperuniformity_test.hyperuniformity_class(k_norm_stop=0.5)
print(sf_power_decay)

In [ ]:
#k_norm = hyperuniformity_test.k_norm
#sf = hyperuniformity_test.sf
sf = s_bi
fitted_poly = hyperuniformity_test.fitted_poly
x = np.linspace(0, 2, 300)
y = np.linspace(0,np.max(k_norm), 500)
fig=plt.figure(figsize=(10,6))
plt.plot(k_norm, sf, 'b.', label="approx_sf")
plt.plot(k_norm, sf, 'b.', label="approx_sf")

plt.plot(k_norm, sf, 'b', label="approx_sf")
plt.plot(y, sf_th_exact(y), 'g', label="exact sf")
plt.plot(x, fitted_poly(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength ($||\mathbf{k}||$)')
plt.ylabel('Structure factor ($\mathsf{S}(\mathbf{k})$)')
plt.show()
#fig.savefig("fitted_si_ginibre.pdf", bbox_inches="tight")

## 4.4- Statistics

In [ ]:
# load points
import pickle
path = "/Users/dhawat/github/structure_factor_paper/test_data/list_samples_thomas_pp.pkl"
with open(path, "rb") as input_file:
    list_sample_thomas_pp = pickle.load(input_file)
    
from structure_factor.spatial_windows import BallWindow, BoxWindow
list_thomas_pp_ball = list_sample_thomas_pp["list_samples_thomas_pp_ball"]
thomas_pp_ball = list_thomas_pp_ball[1]
thomas_pp_ball.plot( file_name="thomas_pp_ball.pdf", window_res=BallWindow(radius=50, center=[0,0]) )

list_thomas_pp_box = list_sample_thomas_pp["list_samples_thomas_pp_box"]
thomas_pp_box = list_thomas_pp_box[1]
thomas_pp_box.plot(file_name="thomas_pp_box.pdf")

In [ ]:
# save samples
# save sample s_bi:
#import pickle
#my_dict = {"list_samples_thomas_pp_ball": thomas_box_pp_list, "list_samples_thomas_pp_box":thomas_box_pp_list}
#with open('list_samples_thomas_pp.pkl', 'wb') as f:
#       pickle.dump(my_dict, f)

In [ ]:
# initialize statistics
from structure_factor.statistics import SummaryStatistics, get_k, get_k_norm
from structure_factor.point_process import ThomasProintProcess

thomas_ball_statistics = SummaryStatistics(list_thomas_pp_ball)
thomas_box_statistics = SummaryStatistics(list_thomas_pp_box)

thomas = ThomasProintProcess( d=2, intensity_parent=1/(20*np.pi), mu=20, sigma=2)

### 4.4.1- For DSE

In [ ]:
from structure_factor.statistics import apply_estimator
from multiprocessing import Pool
from functools import partial
from structure_factor.tapers import SineTaper
import time

start_time = time.time()
core_num=8

k_allowed = get_k(ginibre_box_pp_list[0], k_max=2)
#x_neg = np.linspace(-np.max(np.abs(k_allowed)), -np.min(np.abs(k_allowed)), int(46/2))
x = np.linspace(np.min(np.abs(k_allowed)), np.max(np.abs(k_allowed)), 30)
#x = np.row_stack((x_neg, x_pos)).ravel()
#x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
p = [1, 1]
taper = SineTaper(p)

if __name__ == "__main__":
    # scattering intensity
    s_si_allowed_k = thomas_box_statistics.sample_approximation(estimator="scattering_intensity", k=k_allowed)
    # scattering intensity debiased
    s_dtp_t0 = thomas_box_statistics.sample_approximation(estimator="scattering_intensity", k=k, debiased=True, direct=True)
    # debiased tapered periodogram (sinusoidal taper)
    s_dtp_t1 = thomas_box_statistics.sample_approximation(estimator="tapered_periodogram", k=k,taper=taper, debiased=True, direct=True)
    # multitapered periodogram (sinusoidal taper)
    s_mdtp = thomas_box_statistics.sample_approximation(estimator="multitapered_periodogram", k=k, debiased=True, direct=True)
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
np.max(utils.norm_k(k))

In [ ]:
thomas = ThomasProintProcess( d=2, intensity_parent=1/(20*np.pi), mu=20, sigma=2)
k_norm_si, m_si, ivar_si, ibias_si,imse_si, imse_var_si = sample_statistics(k=k_allowed,
                                                                                approximation=s_si_allowed_k,
                                                                                exact=thomas.structure_factor )
k_norm_dtp_t0, m_dtp_t0, ivar_dtp_t0, ibias_dtp_t0, imse_dtp_t0, imse_var_dtp_t0 = sample_statistics(
                                                                                k=k,
                                                                                approximation=s_dtp_t0,
                                                                                exact=thomas.structure_factor )
k_norm_dtp_t1, m_dtp_t1, ivar_dtp_t1, ibias_dtp_t1, imse_dtp_t1, imse_var_dtp_t1 = sample_statistics(k=k,
                                                                                approximation=s_dtp_t1,
                                                                                exact=thomas.structure_factor)
k_norm_mdtp, m_mdtp, ivar_mdtp, ibias_mdtp, imse_mdtp, imse_var_mdtp = sample_statistics(k=k,
                                                                                approximation=s_mdtp,
                                                                                exact=thomas.structure_factor)
print("For s_si: ivar", ivar_si, "ibias", ibias_si, "imse", imse_si, "3std(imse)",  np.sqrt(imse_var_si)*3)
print("For s_dtp_t0: ivar", ivar_dtp_t0, "ibias", ibias_dtp_t0, "imse", imse_dtp_t0, "3std(imse)",  np.sqrt(imse_var_dtp_t0)*3)
print("For s_dtp_t1: ivar", ivar_dtp_t1, "ibias", ibias_dtp_t1, "imse", imse_dtp_t1, "3std(imse)",  np.sqrt(imse_var_dtp_t1)*3)
print("For s_mdtp: ivar", ivar_mdtp, "ibias", ibias_mdtp, "imse", imse_mdtp, "3std(imse)",  np.sqrt(imse_var_mdtp)*3)

In [ ]:
norm_k_allowed1 = utils.norm_k(k_allowed)
norm_k = utils.norm_k(k)
fig, axis = plt.subplots(figsize=(7,6))

axis.loglog(k_norm_si, m_si, 'b^', label=r"$\overline{\widehat{S}_{\mathrm{SI}}(2\pi \mathbf{n}/L)}$")
#axis.errorbar(k_norm_si, m_si, yerr=var_si, lw=1, ecolor='b', capsize=3, capthick=1)
axis.loglog(k_norm_dtp_t0, m_dtp_t0, 'k*', label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_0, \mathbf{k})}$")
#axis.errorbar(k_norm_dtp_t0, m_dtp_t0, yerr=var_dtp_t0, lw=1, ecolor='k', capsize=3, capthick=1)
axis.loglog(k_norm_dtp_t1, m_dtp_t1, 'm+',label=r"$\overline{\widehat{S}_{\mathrm{DDTP}}(t_1, \mathbf{k})}$")
#axis.errorbar(k_norm_dtp_t1, m_dtp_t1, yerr=var_dtp_t1, lw=1, ecolor='m', capsize=3, capthick=1)
axis.loglog(k_norm_mdtp, m_mdtp, 'c.',  label=r"$\overline{\widehat{S}_{\mathrm{DDMTP}}((t_j)_{j=1}^4, \mathbf{k})}$")
utils.plot_exact(norm_k_allowed1, thomas.structure_factor(norm_k_allowed1), axis=axis, label=r"$S(\mathbf{k})$")
axis.legend()
axis.set_xlabel(r"Wavenumber ($k$)")
axis.set_ylabel(r"Structure factor ($S(k)$)")
fig.savefig("comparaison_DSE_mean_thomas.pdf", bbox_inches="tight")

### 4.4.2- For ISE

In [ ]:
from structure_factor.statistics import apply_estimator
from multiprocessing import Pool
from functools import partial
from structure_factor.statistics import pcf_interpolate_list
import time

start_time = time.time()
core_num=8

n_allowed_k_norm=64
k_norm_allowed = get_k_norm(list_thomas_pp_ball[0], n_allowed_k_norm=n_allowed_k_norm)

if __name__ == "__main__":
    # Bartlett's isotropic estimator
    s_bi = thomas_ball_statistics.sample_approximation(estimator="bartlett_isotropic_estimator", n_allowed_k_norm=n_allowed_k_norm)
    # list pcf
    pcf_list = thomas_ball_statistics.sample_pcf_approximation(method="fv", Kest=dict(rmax=r_max), fv=dict(method="b", spar=0.1))
    
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
s = len(pcf_list)
# interpolate pcf
r_list = [pcf_list[i]["r"] for i in range(s)]
pcf_fv_list = [pcf_list[i]["pcf"] for i in range(s)]
pcf_fct_list = pcf_interpolate_list(r_list, pcf_fv_list, drop=True)
# BaddourChouinard
s_hbc_list = thomas_ball_statistics.sample_integral_approximation(pcf_fct_list,method="BaddourChouinard", 
                                                                   k_norm=k_norm_allowed, r_max=r_max,  nb_points=1000) 
# Ogata
s_ho_list = thomas_ball_statistics.sample_integral_approximation(pcf_fct_list,method="Ogata", 
                                                                   k_norm=k_norm_allowed, step_size=0.01,  nb_points=1000) 


In [ ]:
k_bi, m_bi, ivar_bi, ibias_bi, imse_bi, imse_var_bi = sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_bi,
                                                                                exact=thomas.structure_factor )
k_hbc, m_hbc, ivar_hbc, ibias_hbc, imse_hbc, imse_var_hbc = sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_hbc_list,
                                                                                exact=thomas.structure_factor)
k_ho, m_ho, ivar_ho, ibias_ho, imse_ho, imse_var_ho = sample_statistics(k_norm=k_norm_allowed,
                                                                                approximation=s_ho_list,
                                                                                exact=thomas.structure_factor)
print("For s_bi: ivar=", ivar_bi, "ibias=", ibias_bi, "imse=", imse_bi, "3std(imse)",  np.sqrt(imse_var_bi)*3)

print("For s_hbc: ivar=", ivar_hbc, "ibias=", ibias_hbc, "imse=", imse_hbc, "3std(imse)",  np.sqrt(imse_var_hbc)*3)

print("For s_ho: ivar=", ivar_ho, "ibias=", ibias_ho, "imse=", imse_ho, "3std(imse)",  np.sqrt(imse_var_ho)*3)


In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
axis.loglog(k_norm_allowed, m_bi, 'm*', label=r"$\overline{\widehat{S}_{\mathrm{BI}}(k)}$")
#axis.loglog(k_norm_allowed, m_bi, 'm')
axis.loglog(k_ho, m_ho, 'b.', label=r"$\overline{\widehat{S}_{\mathrm{HO}}(k)}$")
#axis.loglog(k_norm_allowed, m_ho, 'b')
axis.loglog(k_hbc, m_hbc, 'k+', label=r"$\overline{\widehat{S}_{\mathrm{HBC}}(k)}$")
#axis.loglog(k_norm_allowed, m_hbc, 'k')
axis.loglog(k_norm_allowed, thomas.structure_factor(k_norm_allowed), 'g', label= r"$S(k)$")
axis.legend()
axis.set_xlabel(r"Wavenumber ($k$)")
axis.set_ylabel(r"Structure factor ($S(k)$)")
plt.show()
fig.savefig("comparaison_ISE_mean_thomas.pdf", bbox_inches="tight")

In [ ]:
fig, axis = plt.subplots(figsize=(7,6))
axis.loglog(k_norm_allowed, m_bi, 'm*', label=r"$\overline{\widehat{S}_{\mathrm{BI}}(k)}$")
axis.loglog(k_norm_allowed, m_bi, 'm')
axis.loglog(k_norm_allowed, m_ho, 'b.', label=r"$\overline{\widehat{S}_{\mathrm{HO}}(k)}$")
#axis.loglog(k_norm_allowed, m_ho, 'b')
axis.loglog(k_norm_allowed, m_hbc, 'k+', label=r"$\overline{\widehat{S}_{\mathrm{HBC}}(k)}$")
#axis.loglog(k_norm_allowed, m_hbc, 'k')
axis.loglog(k_norm_allowed, thomas.structure_factor(k_norm_allowed), 'g', label= r"$S(k)$")
axis.legend()
axis.set_xlabel(r"Wavenumber ($k$)")
axis.set_ylabel(r"Structure factor ($S(k)$)")
plt.show()
fig.savefig("comparaison_ISE_mean_thomas.png", bbox_inches="tight")

## 4.4.3- Hyperuniformity


In [ ]:
# using bartlett's isotropic estimator

from structure_factor.statistics import apply_estimator
from multiprocessing import Pool
from functools import partial
from structure_factor.statistics import pcf_interpolate_list
import time

start_time = time.time()
core_num=8

n_allowed_k_norm=64
k_norm_allowed = get_k_norm(thomas_ball_pp_list[0], n_allowed_k_norm=n_allowed_k_norm)

if __name__ == "__main__":
    # Bartlett's isotropic estimator
    s_bi = thomas_ball_statistics.sample_approximation(estimator="bartlett_isotropic_estimator", n_allowed_k_norm=n_allowed_k_norm)
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# save sample s_bi:
import pickle
my_dict = {"k_norm": k_norm_allowed, "list_s_bi_thomas": s_bi}
with open('sample_s_bi_thomas.pkl', 'wb') as f:
       pickle.dump(my_dict, f)

In [ ]:
plt.plot(k_norm_allowed, s_bi[5], 'b.')

### 4.4.3.1- effective hyperuniformity

In [ ]:
from structure_factor.statistics import sample_hyperuniformity, sample_variance

H_index_list = sample_hyperuniformity(k_norm_allowed, s_bi, estimator="effective", k_norm_stop=0.5)
H_var, H_m = sample_variance(H_index_list)
print("mean H index", H_m, "3std H index", 3*np.sqrt(H_var))